# Title
[]()

In [1]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

# Dataframe Display Parameters

In [325]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
from pypdf import PdfReader
import pandas as pd
import openai
import os
from openai.embeddings_utils import get_embedding, cosine_similarity
import re

# Load CSV

In [293]:
filename = 'export knowledge library 2023-04-24.csv'
path = r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\sources'
kl = load_csv(filename, path).dropna(subset=["Content: Silvia's version Part 1"], how='all')
kl

Dataframe shape:  (1995, 90)
DataFrame columns: ['Knowledge ID #', 'Title', 'Date added', 'Date revised', 'social media', 'Citation 1', 'Citation 2', 'Citation 3', 'Reference saved', 'Reference saved.1', 'Reference saved.2', "Silvia's comments re: saving references", "Content: Dylan/Rob's notification", "Content: Dylan/Rob's Expanded Notification", "Content: Dylan/Rob's Read More", "Content: Silvia's notification", "Content: Silvia's version Part 1", "Content: Silvia's version Part 2", 'Link to other content in library', 'Level', 'Age: Lower', 'Age: Upper', 'User: Sex', 'User: Has children', 'User: caretaker of adult', 'Smoker', 'Notification Time', 'Domain: Movement', 'Domain: Nutrition', 'Domain: Medical', 'Domain: Sleep', 'Domain: Emotional health', 'Domain: Brain', 'Domain: Injury', 'Domain: Social', 'Domain: Behaviour change', 'Domain: Cooking', 'Activity', 'Domain: Subdomain ', 'Link to Associated challenge', 'Relevant to another part of the platform', 'combination', 'Tags >', 'P

,Knowledge ID #,Title,Date added,Date revised,social media,Citation 1,Citation 2,Citation 3,Reference saved,Reference saved.1,Reference saved.2,Silvia's comments re: saving references,Content: Dylan/Rob's notification,Content: Dylan/Rob's Expanded Notification,Content: Dylan/Rob's Read More,Content: Silvia's notification,Content: Silvia's version Part 1,Content: Silvia's version Part 2,Link to other content in library,Level,Age: Lower,Age: Upper,User: Sex,User: Has children,User: caretaker of adult,Smoker,Notification Time,Domain: Movement,Domain: Nutrition,Domain: Medical,Domain: Sleep,Domain: Emotional health,Domain: Brain,Domain: Injury,Domain: Social,Domain: Behaviour change,Domain: Cooking,Activity,Domain: Subdomain,Link to Associated challenge,Relevant to another part of the platform,combination,Tags >,Population: General,Population: BMI <20 or eating disorder,Population: Interest in fat loss or weight control,Population: Muscle gain,Population: Overweight & obesity,Population: Type 1 Diabetes,Population: type 2 Diabetes,Population: High BP,Population: Pulmonary High BP,Population: CVD,Population: Metabolic Syndrome,Population: Coronary Artery Disease,Population: Peripheral Artery Disease,Population: Dyslipidemia,Population: Chronic Heart Failure,Population: Arrhythmia,Population: depression,Population: Anxiety disorder,Population: Osteopenia & osteoporosis,Population: balance or walking difficulties,Population: Stroke,Population: Parkinson’s Disease,Population: multiple sclerosis,Population: Arthritis,Population: Sarcopenia,Population: age-related cognitive decline / dementia,Population: Chronic Kidney Disease,Population: sleep apnea,Population: Asthma,Population: COPD,Population: Cystic Fibrosis,Population: Autoimmune Disorder,Population: cancer: general,Population: Cancer: Breast,Population: Cancer: Colorectal,Population: Cancer: Bronchogenic / Lung,Population: Cancer: Head and/or neck,"Population: Cancer: Metastatic / Pelvis, legs, or spine",Population: Cancer: Multiple myeloma,Population: Cancer: Prostate,Population: Cancer: Skin,Population: HIV,Population: Down syndrome,Population: Concussion,Population: chronic low back pain,Population: wheelchair,Population: spinal cord injury
0,1.0,exercises for belly fat,2021-07-06,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Can you target belly fat?,You can’t specifically target belly fat with exercise - instead taking an approach that allows f...,This old myth lives on for marketing purposes - not for practical and helpful ones. If you want ...,Exercises for losing fat from the belly (or other stubborn areas).,Instead of doing a ton of ab exercises to get rid of the belly or using the thigh master slim do...,Doing abdominal exercises may not help with losing belly fat. While we may want to lose fat from...,waist to height ratio; waist circumference and WHR; 1 inch waist reduction; cortisol and abdomin...,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Exercises for losing fat from the belly (or other stubborn areas). [part 1] Instead of doing a t...,NaN,y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Why bedtime matters for blood sugar,2022-01-17,NaN,NaN,"Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Lev...",Examine. (2022 January). Sleep characteristics influence glucose regulation the next day . Exami...,"Tsereteli, N., Vallat, R., Fernandez-Tajes, J., Delahanty, L. M., Ordovas, J. M., Drew, D. A., V...",Not available,not found,saved,Unable to find the Examine research summary despite searching their website and Google.,Your sleep can impact blood sugar,Getting enough sleep combined with getting to bed at a decent time can help improve your glucose...,Being able to clear blood sugar efficiently is important for reducing your risk for type 2 diabe...,NaN,Ai

In [5]:
references = kl[['Knowledge ID #', 'Title', 'Citation 1', 'Citation 2', 'Citation 3']]
references

,Knowledge ID #,Title,Citation 1,Citation 2,Citation 3
0,1.0,exercises for belly fat,NaN,NaN,NaN
1,2.0,Why bedtime matters for blood sugar,"Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255","Examine. (2022 January). Sleep characteristics influence glucose regulation the next day . Examine Study Summaries, https://examine.com/members/summaries/issue/january-2022/summary-1lGLqd","Tsereteli, N., Vallat, R., Fernandez-Tajes, J., Delahanty, L. M., Ordovas, J. M., Drew, D. A., Valdes, A. M., Segata, N., Chan, A. T., Wolf, J., Berry, S. E., Walker, M. P., Spector, T. D., & Franks, P. W. (2022). Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions. Diabetologia, 65(2), 356–365. https://doi.org/10.1007/s00125-021-05608-y"
2,3.0,DOMS reduces motor learning,"Zourdos, M.C. (2020). We Out Here Talkin’ About Practice, Man. Monthly Applications in Strength Sport, 4(1), 71-80.",NaN,NaN
3,4.0,adjust training instead of skipping it,"Helms, E.. (2019). Give Better Guidance than “Just Work Harder”. Monthly Applications in Strength Sport, 3(6), 57-65.",NaN,NaN
4,5.0,habits are important for fat loss maintenance,"Eiselt, A. (2019). How the Brain Controls Eating Behavior. Monthly Applications in Strength Sport, 3(12), 58-69.",NaN,NaN
...,...,...,...,...,...
1212,1213.0,climate impact of ultra processed food,"Lennon, D. (Host). (2022 July 13). How Climate Change Impacts Nutrient Status (No. 446) [Audio podcast episode]. In Sigma Nutrition Radio. Sigma Nutrition. https://sigmanutrition.com/episode446/","Hallström, E., Bajzelj, B., Håkansson, N., Sjons, J., Åkesson, A., Wolk, A., & Sonesson, U. (2021). Dietary climate impact: Contribution of foods and dietary patterns by gender and age in a Swedish population. Journal of Cleaner Production, 306, 127189.",NaN
1213,1214.0,menopause increases cardiovascular disease,"Lennon, D. (Host). (2022). Nutrition for Women’s Health (No. 445) [Audio podcast episode]. In Sigma Nutrition Radio. Sigma Nutrition. https://sigmanutrition.com/episode445/",NaN,NaN
1214,1215.0,Plant biodiversity enhances nature's effect on mental health,Optimal Aging Portal Blog Team. (2021 February 24). Trees in the city: the roots of health and well-being. McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2021/02/24/trees-in-the-city-the-roots-of-health-and-well-being?sid=f276c79e-05c9-4086-ada5-3079d5b35086&hl=gardening,NaN,NaN
1215,1216.0,Therapeutic gardening\r,Horticultural therapy: Are there benefits to guided gardening?. (2022 August 3). McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2022/08/03/horticultural-therapy-are-there-benefits-to-guided-gardening,"Horticultural therapy for general health in the older adults: A systematic review and meta-analysis. (n.d.). Retrieved August 19, 2022 from McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/full-article/he/horticultural-therapy-general-health-older-adults-systematic-review-meta-42628?hl=horticultural+therapy",NaN


In [43]:
references.loc[142]

Knowledge ID #                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [44]:
references.loc[205]

Knowledge ID #                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

# Join references

## Experiment with joining text

In [37]:
pd.DataFrame(references[['Citation 1', 'Citation 2', 'Citation 3']].fillna('').apply(lambda x: ''.join(x.astype(str)+'\n'), axis=1), index=references['Knowledge ID #'])

,0
Knowledge ID #,
1.0,"Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255\nExamine. (2022 January). Sleep characteristics influence glucose regulation the next day . Examine Study Summaries, https://examine.com/members/summaries/issue/january-2022/summary-1lGLqd\nTsereteli, N., Vallat, R., Fernandez-Tajes, J., Delahanty, L. M., Ordovas, J. M., Drew, D. A., Valdes, A. M., Segata, N., Chan, A. T., Wolf, J., Berry, S. E., Walker, M. P., Spector, T. D., & Franks, P. W. (2022). Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions. Diabetologia, 65(2), 356–365. https://doi.org/10.1007/s00125-021-05608-y\n"
2.0,"Zourdos, M.C. (2020). We Out Here Talkin’ About Practice, Man. Monthly Applications in Strength Sport, 4(1), 71-80.\n\n\n"
3.0,"Helms, E.. (2019). Give Better Guidance than “Just Work Harder”. Monthly Applications in Strength Sport, 3(6), 57-65.\n\n\n"
4.0,"Eiselt, A. (2019). How the Brain Controls Eating Behavior. Monthly Applications in Strength Sport, 3(12), 58-69.\n\n\n"
5.0,"Helms, E. (2018). Visualize Your Way to Gains: Dose Response of Mental Imagery for Strength. Monthly Applications in Strength Sport, 2(5), 68-78.\nParavlic, A. H., Slimani, M., Tod, D., Marusic, U., Milanovic, Z., & Pisot, R. (2018). Effects and Dose-Response Relationships of Motor Imagery Practice on Strength Development in Healthy Adult Populations: a Systematic Review and Meta-analysis. Sports medicine (Auckland, N.Z.), 48(5), 1165–1187. https://doi.org/10.1007/s40279-018-0874-8\n\n"
...,...
1213.0,"Lennon, D. (Host). (2022). Nutrition for Women’s Health (No. 445) [Audio podcast episode]. In Sigma Nutrition Radio. Sigma Nutrition. https://sigmanutrition.com/episode445/\n\n\n"
1214.0,Optimal Aging Portal Blog Team. (2021 February 24). Trees in the city: the roots of health and well-being. McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2021/02/24/trees-in-the-city-the-roots-of-health-and-well-being?sid=f276c79e-05c9-4086-ada5-3079d5b35086&hl=gardening\n\n\n
1215.0,"Horticultural therapy: Are there benefits to guided gardening?. (2022 August 3). McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2022/08/03/horticultural-therapy-are-there-benefits-to-guided-gardening\nHorticultural therapy for general health in the older adults: A systematic review and meta-analysis. (n.d.). Retrieved August 19, 2022 from McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/full-article/he/horticultural-therapy-general-health-older-adults-systematic-review-meta-42628?hl=horticultural+therapy\n\n"


In [38]:
references[['Citation 1', 'Citation 2', 'Citation 3']].fillna('').apply(lambda x: ''.join(x.astype(str)+'\n'), axis=1)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                \n\n\n
1       Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https

In [39]:
references[['Citation 1', 'Citation 2', 'Citation 3']].fillna('').apply(lambda x: ''.join(x.astype(str)+'\n'), axis=1).str.split('\n')

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                [, , , ]
1       [Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification ht

In [42]:
references[['Citation 1', 'Citation 2', 'Citation 3']].fillna('').apply(lambda x: ''.join(x.astype(str)+'\n'), axis=1).str.split('\n').str.len().sort_values(ascending=False)

142     14
205     14
175     12
344     11
179     11
        ..
488      4
489      4
490      4
491      4
1216     4
Length: 1217, dtype: int64

In [47]:
references[['Citation 1', 'Citation 2', 'Citation 3']].fillna('').apply(lambda x: ''.join(x.astype(str)+'\n'), axis=1).str.replace(',', '[comma]').str.split('\n')

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [48]:
references[['Citation 1', 'Citation 2', 'Citation 3']].fillna('').apply(lambda x: ''.join(x.astype(str)+'\n'), axis=1).str.replace(',', '[comma]').str.split('\n').str.len().sort_values(ascending=False)

142     14
205     14
175     12
344     11
179     11
        ..
488      4
489      4
490      4
491      4
1216     4
Length: 1217, dtype: int64

In [49]:
references[['Citation 1', 'Citation 2', 'Citation 3']].fillna('').apply(lambda x: ''.join(x.astype(str)+'\n'), axis=1).str.replace(',', '[comma]').str.split('\n').loc[142]

['Suttie[comma] J. (2017 November 13). Four Ways Social Support Makes You More Resilient. Greater Good Magazine. https://greatergood.berkeley.edu/article/item/four_ways_social_support_makes_you_more_resilient',
 '',
 'Baxter[comma] J.[comma] She’ Ierly[comma] S. M.[comma] Eby[comma] C.[comma] Mason[comma] L.[comma] Cortese[comma] C. F.[comma] & Hamman[comma] R. F. (1998). Social Network Factors Associated with Perceived Quality of Life: The San Luis Valley Health and Aging Study. Journal of Aging and Health[comma] 10(3)[comma] 287–310. https://doi.org/10.1177/089826439801000302',
 '',
 'Chan[comma] Y. K.[comma] & Lee[comma] R. P. (2006). Network size[comma] social support and happiness in later life: A comparative study of Beijing and Hong Kong. Journal of Happiness Studies[comma] 7(1)[comma] 87-112.',
 '',
 'Heinze[comma] J. E.[comma] Kruger[comma] D. J.[comma] Reischl[comma] T. M.[comma] Cupal[comma] S.[comma] & Zimmerman[comma] M. A. (2015). Relationships Among Disease[comma] Social

In [51]:
references[['Citation 1', 'Citation 2', 'Citation 3']].fillna('').apply(lambda x: ''.join(x.astype(str)+'\n'), axis=1).str.replace('\n\n', '\n').str.replace(',', '[comma]').str.split('\n')

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [60]:
references[['Citation 1', 'Citation 2', 'Citation 3']].fillna('').apply(lambda x: ''.join(x.astype(str)+'\n'), axis=1).str.replace('\n\n', '\n').str.replace(',', '[comma]').str.split('\n').str.len().sort_values(ascending=False)

205     13
175     10
211      9
739      9
656      9
        ..
489      3
490      3
491      3
492      3
1216     3
Length: 1217, dtype: int64

In [67]:
references[['Citation 1', 'Citation 2', 'Citation 3']].fillna('').apply(lambda x: ''.join(x.astype(str)+'\n'), axis=1).str.replace(r'[\n\r]{2}', '\n', regex=True).str.replace(',', '[comma]').str.split('\n').str.len().sort_values(ascending=False)

205    13
175    10
211     9
236     9
430     9
       ..
184     3
648     3
183     3
650     3
0       3
Length: 1217, dtype: int64

In [ ]:
references[['Citation 1', 'Citation 2', 'Citation 3']].fillna('').apply(lambda x: ''.join(x.astype(str)+'\n'), axis=1).str.replace(r'[\n\r]{2}', '\n', regex=True).str.replace(',', '[comma]').str.split('\n').str.len().sort_values(ascending=False)

205    13
175    10
211     9
236     9
430     9
       ..
184     3
648     3
183     3
650     3
0       3
Length: 1217, dtype: int64

In [68]:
references[['Citation 1', 'Citation 2', 'Citation 3']].fillna('').apply(lambda x: ''.join(x.astype(str)+'\n'), axis=1).str.replace('\n\n', '\n').str.replace(',', '[comma]').str.split('\n').loc[205]

['\t1. Khoury[comma] B.[comma] Lecomte[comma] T.[comma] Fortin[comma] G.[comma] Masse[comma] M.[comma] Therien[comma] P.[comma] Bouchard[comma] V.[comma] Chapleau[comma] M. A.[comma] Paquin[comma] K.[comma] & Hofmann[comma] S. G. (2013). Mindfulness-based therapy: a comprehensive meta-analysis. Clinical psychology review[comma] 33(6)[comma] 763–771. https://doi.org/10.1016/j.cpr.2013.05.005\r',
 '\t2. Keng[comma] S. L.[comma] Smoski[comma] M. J.[comma] & Robins[comma] C. J. (2011). Effects of mindfulness on psychological health: a review of empirical studies. Clinical psychology review[comma] 31(6)[comma] 1041–1056. https://doi.org/10.1016/j.cpr.2011.04.006\r',
 '\tFriese[comma] M.[comma] Messner[comma] C.[comma] & Schaffner[comma] Y. (2012). Mindfulness meditation counteracts self-control depletion. Consciousness and Cognition[comma] 21(2)[comma] 1016-1022. doi:10.1016/j.concog.2012.01.008 [doi]\r',
 '\t3. Mohammed[comma] W. A.[comma] Pappous[comma] A.[comma] & Sharma[comma] D. (2018)

In [62]:
references[['Citation 1', 'Citation 2', 'Citation 3']].fillna('').apply(lambda x: ''.join(x.astype(str)+'\n'), axis=1).str.replace(r'[\n\r]{2}', '\n').str.replace(',', '[comma]').str.split('\n').loc[205]

C:\Users\silvh\AppData\Local\Temp\ipykernel_25016\1560491489.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  references[['Citation 1', 'Citation 2', 'Citation 3']].fillna('').apply(lambda x: ''.join(x.astype(str)+'\n'), axis=1).str.replace(r'[\n\r]{2}', '\n').str.replace(',', '[comma]').str.split('\n').loc[205]


['\t1. Khoury[comma] B.[comma] Lecomte[comma] T.[comma] Fortin[comma] G.[comma] Masse[comma] M.[comma] Therien[comma] P.[comma] Bouchard[comma] V.[comma] Chapleau[comma] M. A.[comma] Paquin[comma] K.[comma] & Hofmann[comma] S. G. (2013). Mindfulness-based therapy: a comprehensive meta-analysis. Clinical psychology review[comma] 33(6)[comma] 763–771. https://doi.org/10.1016/j.cpr.2013.05.005',
 '\t2. Keng[comma] S. L.[comma] Smoski[comma] M. J.[comma] & Robins[comma] C. J. (2011). Effects of mindfulness on psychological health: a review of empirical studies. Clinical psychology review[comma] 31(6)[comma] 1041–1056. https://doi.org/10.1016/j.cpr.2011.04.006',
 '\tFriese[comma] M.[comma] Messner[comma] C.[comma] & Schaffner[comma] Y. (2012). Mindfulness meditation counteracts self-control depletion. Consciousness and Cognition[comma] 21(2)[comma] 1016-1022. doi:10.1016/j.concog.2012.01.008 [doi]',
 '\t3. Mohammed[comma] W. A.[comma] Pappous[comma] A.[comma] & Sharma[comma] D. (2018). Effe

# Isolate the references

In [129]:
references_lists = pd.DataFrame(
    references[['Citation 1', 'Citation 2', 'Citation 3']].fillna('').apply(
    lambda x: ''.join(x.astype(str)+'\n' ), axis=1).str.replace('\n\n', '\n').str.replace(',', '[comma]').str.replace('\t\n','').str.split('\n')
)
# references_lists = reference for reference in references_lists
references_lists

,0
0,"[, , ]"
1,"[Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17[comma] 2022 from PN Level 1 Sleep[comma] Stress Management[comma] and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255, Examine. (2022 January). Sleep characteristics influence glucose regulation the next day . Examine Study Summaries[comma] https://examine.com/members/summaries/issue/january-2022/summary-1lGLqd, Tsereteli[comma] N.[comma] Vallat[comma] R.[comma] Fernandez-Tajes[comma] J.[comma] Delahanty[comma] L. M.[comma] Ordovas[comma] J. M.[comma] Drew[comma] D. A.[comma] Valdes[comma] A. M.[comma] Segata[comma] N.[comma] Chan[comma] A. T.[comma] Wolf[comma] J.[comma] Berry[comma] S. E.[comma] Walker[comma] M. P.[comma] Spector[comma] T. D.[comma] & Franks[comma] P. W. (2022). Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions. Diabetologia[comma] 65(2)[comma] 356–365. https://doi.org/10.1007/s00125-021-05608-y, ]"
2,"[Zourdos[comma] M.C. (2020). We Out Here Talkin’ About Practice[comma] Man. Monthly Applications in Strength Sport[comma] 4(1)[comma] 71-80., , ]"
3,"[Helms[comma] E.. (2019). Give Better Guidance than “Just Work Harder”. Monthly Applications in Strength Sport[comma] 3(6)[comma] 57-65., , ]"
4,"[Eiselt[comma] A. (2019). How the Brain Controls Eating Behavior. Monthly Applications in Strength Sport[comma] 3(12)[comma] 58-69., , ]"
...,...
1212,"[Lennon[comma] D. (Host). (2022 July 13). How Climate Change Impacts Nutrient Status (No. 446) [Audio podcast episode]. In Sigma Nutrition Radio. Sigma Nutrition. https://sigmanutrition.com/episode446/, Hallström[comma] E.[comma] Bajzelj[comma] B.[comma] Håkansson[comma] N.[comma] Sjons[comma] J.[comma] Åkesson[comma] A.[comma] Wolk[comma] A.[comma] & Sonesson[comma] U. (2021). Dietary climate impact: Contribution of foods and dietary patterns by gender and age in a Swedish population. Journal of Cleaner Production[comma] 306[comma] 127189., ]"
1213,"[Lennon[comma] D. (Host). (2022). Nutrition for Women’s Health (No. 445) [Audio podcast episode]. In Sigma Nutrition Radio. Sigma Nutrition. https://sigmanutrition.com/episode445/, , ]"
1214,"[Optimal Aging Portal Blog Team. (2021 February 24). Trees in the city: the roots of health and well-being. McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2021/02/24/trees-in-the-city-the-roots-of-health-and-well-being?sid=f276c79e-05c9-4086-ada5-3079d5b35086&hl=gardening, , ]"
1215,"[Horticultural therapy: Are there benefits to guided gardening?. (2022 August 3). McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2022/08/03/horticultural-therapy-are-there-benefits-to-guided-gardening, Horticultural therapy for general health in the older adults: A systematic review and meta-analysis. (n.d.). Retrieved August 19[comma] 2022 from McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/full-article/he/horticultural-therapy-general-health-older-adults-systematic-review-meta-42628?hl=horticultural+therapy, ]"


In [130]:
references_lists.loc[175]

0    [1. [see "how to use social media for mental health" in knowledge library], \t2. Andreassen[comma] C. S.[comma] Torsheim[comma] T.[comma] & Pallesen[comma] S. (2014). Use of Online Social Network Sites for Personal Purposes at Work: Does it Impair Self-Reported Performance? Comprehensive Psychology. https://doi.org/10.2466/01.21.CP.3.18, \t3. Schmidt[comma] S. J. (2020). Distracted learning: Big problem and golden opportunity. Journal of Food Science Education[comma] 19(4)[comma] 278-291., \tUniversity of Illinois at Urbana-Champaign[comma] News Bureau. (2020 October 14). Distracted learning a big problem[comma] golden opportunity for educators[comma] students. Science Daily. https://www.sciencedaily.com/releases/2020/10/201014140932.htm, \t4. Simon Fraser University. (2019 October 3). Addictive de-vices: How we can unplug from this 21st century epidemic. Science Daily. https://www.sciencedaily.com/releases/2019/10/191003092039.htm, \tBerthon[comma] P.[comma] Pitt[comma] L.[comma]

# Create final dataframe with separated references

In [131]:
separated_references_original = pd.concat([pd.Series(references_lists[0].iloc[i], index=range(len(references_lists[0].iloc[i]))) for i in range(len(references_lists))], axis=1).T.dropna(axis=1, how='all')
separated_references_original

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17[comma] 2022 from PN Level 1 Sleep[comma] Stress Management[comma] and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255,Examine. (2022 January). Sleep characteristics influence glucose regulation the next day . Examine Study Summaries[comma] https://examine.com/members/summaries/issue/january-2022/summary-1lGLqd,Tsereteli[comma] N.[comma] Vallat[comma] R.[comma] Fernandez-Tajes[comma] J.[comma] Delahanty[comma] L. M.[comma] Ordovas[comma] J. M.[comma] Drew[comma] D. A.[comma] Valdes[comma] A. M.[comma] Segata[comma] N.[comma] Chan[comma] A. T.[comma] Wolf[comma] J.[comma] Berry[comma] S. E.[comma] Walker[comma] M. P.[comma] Spector[comma] T. D.[comma] & Franks[comma] P. W. (2022). Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions. Diabetologia[comma] 65(2)[comma] 356–365. https://doi.org/10.1007/s00125-021-05608-y,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Zourdos[comma] M.C. (2020). We Out Here Talkin’ About Practice[comma] Man. Monthly Applications in Strength Sport[comma] 4(1)[comma] 71-80.,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Helms[comma] E.. (2019). Give Better Guidance than “Just Work Harder”. Monthly Applications in Strength Sport[comma] 3(6)[comma] 57-65.,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Eiselt[comma] A. (2019). How the Brain Controls Eating Behavior. Monthly Applications in Strength Sport[comma] 3(12)[comma] 58-69.,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1212,Lennon[comma] D. (Host). (2022 July 13). How Climate Change Impacts Nutrient Status (No. 446) [Audio podcast episode]. In Sigma Nutrition Radio. Sigma Nutrition. https://sigmanutrition.com/episode446/,Hallström[comma] E.[comma] Bajzelj[comma] B.[comma] Håkansson[comma] N.[comma] Sjons[comma] J.[comma] Åkesson[comma] A.[comma] Wolk[comma] A.[comma] & Sonesson[comma] U. (2021). Dietary climate impact: Contribution of foods and dietary patterns by gender and age in a Swedish population. Journal of Cleaner Production[comma] 306[comma] 127189.,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1213,Lennon[comma] D. (Host). (2022). Nutrition for Women’s Health (No. 445) [Audio podcast episode]. In Sigma Nutrition Radio. Sigma Nutrition. https://sigmanutrition.com/episode445/,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1214,Optimal Aging Portal Blog Team. (2021 February 24). Trees in the city: the roots of health and well-being. McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2021/02/24/trees-in-the-city-the-roots-of-health-and-well-being?sid=f276c79e-05c9-4086-ada5-3079d5b35086&hl=gardening,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1215,Horticultural therapy: Are there benefits to guided gardening?. (2022 August 3). McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2022/08/03/horticultural-therapy-are-there-benefits-to-guided-gardening,Horticultural therapy for general health in the older adults: A systematic review and meta-analysis. (n.d.). Retrieved August 19[comma] 2022 from McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/full-article/he/horticultural-therapy-general-health-older-adults-systematic-review-meta-42628?hl=horticultural+therapy,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Figure out why there are 13 columns with last 2 empty

In [44]:
separated_references_original.iloc[:, -3:-1]

,Citation 11,Citation 12
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
1212,NaN,NaN
1213,NaN,NaN
1214,NaN,NaN
1215,NaN,NaN


In [58]:
separated_references_original.fillna(0)[separated_references_original['Citation 13'].fillna(0) != 0]

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10,Citation 11,Citation 12,Citation 13
205,\t1. Khoury[comma] B.[comma] Lecomte[comma] T.[comma] Fortin[comma] G.[comma] Masse[comma] M.[comma] Therien[comma] P.[comma] Bouchard[comma] V.[comma] Chapleau[comma] M. A.[comma] Paquin[comma] K.[comma] & Hofmann[comma] S. G. (2013). Mindfulness-based therapy: a comprehensive meta-analysis. Clinical psychology review[comma] 33(6)[comma] 763–771. https://doi.org/10.1016/j.cpr.2013.05.005\r,\t2. Keng[comma] S. L.[comma] Smoski[comma] M. J.[comma] & Robins[comma] C. J. (2011). Effects of mindfulness on psychological health: a review of empirical studies. Clinical psychology review[comma] 31(6)[comma] 1041–1056. https://doi.org/10.1016/j.cpr.2011.04.006\r,\tFriese[comma] M.[comma] Messner[comma] C.[comma] & Schaffner[comma] Y. (2012). Mindfulness meditation counteracts self-control depletion. Consciousness and Cognition[comma] 21(2)[comma] 1016-1022. doi:10.1016/j.concog.2012.01.008 [doi]\r,\t3. Mohammed[comma] W. A.[comma] Pappous[comma] A.[comma] & Sharma[comma] D. (2018). Effect of Mindfulness Based Stress Reduction (MBSR) in Increasing Pain Tolerance and Improving the Mental Health of Injured Athletes. Frontiers in psychology[comma] 9[comma] 722. https://doi.org/10.3389/fpsyg.2018.00722\r,\tUssher[comma] M.[comma] Spatz[comma] A.[comma] Copland[comma] C.[comma] Nicolaou[comma] A.[comma] Cargill[comma] A.[comma] Amini-Tabrizi[comma] N.[comma] & McCracken[comma] L. M. (2014). Immediate effects of a brief mindfulness-based body scan on patients with chronic pain. Journal of behavioral medicine[comma] 37(1)[comma] 127–134. https://doi.org/10.1007/s10865-012-9466-5\r,\t4. Warren[comma] J. M.[comma] Smith[comma] N.[comma] & Ashwell[comma] M. (2017). A structured literature review on the role of mindfulness[comma] mindful eating and intuitive eating in changing eating behaviours: Effectiveness and associated potential mechanisms. Nutrition Research Reviews[comma] 30(2)[comma] 272-283. doi:10.1017/S0954422417000154 [doi]\r,\t5. Nien[comma] J. T.[comma] Wu[comma] C. H.[comma] Yang[comma] K. T.[comma] Cho[comma] Y. M.[comma] Chu[comma] C. H.[comma] Chang[comma] Y. K.[comma] & Zhou[comma] C. (2020). Mindfulness Training Enhances Endurance Performance and Executive Functions in Athletes: An Event-Related Potential Study. Neural plasticity[comma] 2020[comma] 8213710. https://doi.org/10.1155/2020/8213710\r,\t6. Cropley[comma] M.[comma] Ussher[comma] M.[comma] & Charitou[comma] E. (2007). Acute effects of a guided relaxation routine (body scan) on tobacco withdrawal symptoms and cravings in abstinent smokers. Addiction (Abingdon[comma] England)[comma] 102(6)[comma] 989–993. https://doi.org/10.1111/j.1360-0443.2007.01832.x\r,\tBrewer[comma] J. A.[comma] Mallik[comma] S.[comma] Babuscio[comma] T. A.[comma] Nich[comma] C.[comma] Johnson[comma] H. E.[comma] Deleone[comma] C. M.[comma] Minnix-Cotton[comma] C. A.[comma] Byrne[comma] S. A.[comma] Kober[comma] H.[comma] Weinstein[comma] A. J.[comma] Carroll[comma] K. M.[comma] & Rounsaville[comma] B. J. (2011). Mindfulness training for smoking cessation: results from a randomized controlled trial. Drug and alcohol dependence[comma] 119(1-2)[comma] 72–80. https://doi.org/10.1016/j.drugalcdep.2011.05.027\r,\t\r,Berardi[comma] J. (n.d.). Why ‘listen to your body’ and ‘do what works for you’ rarely work — and what to do instead. Retrieved December 13[comma] 2021 from Precision Nutrition. https://www.precisionnutrition.com/listen-to-your-body,,


In [61]:
separated_references_original.loc[205, 'Citation 13']

''

In [37]:
separated_references_original.loc[[205, 142]]

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10,Citation 11,Citation 12,Citation 13
205,"\t1. Khoury, B., Lecomte, T., Fortin, G., Masse, M., Therien, P., Bouchard, V., Chapleau, M. A., Paquin, K., & Hofmann, S. G. (2013). Mindfulness-based therapy: a comprehensive meta-analysis. Clinical psychology review, 33(6), 763–771. https://doi.org/10.1016/j.cpr.2013.05.005\r","\t2. Keng, S. L., Smoski, M. J., & Robins, C. J. (2011). Effects of mindfulness on psychological health: a review of empirical studies. Clinical psychology review, 31(6), 1041–1056. https://doi.org/10.1016/j.cpr.2011.04.006\r","\tFriese, M., Messner, C., & Schaffner, Y. (2012). Mindfulness meditation counteracts self-control depletion. Consciousness and Cognition, 21(2), 1016-1022. doi:10.1016/j.concog.2012.01.008 [doi]\r","\t3. Mohammed, W. A., Pappous, A., & Sharma, D. (2018). Effect of Mindfulness Based Stress Reduction (MBSR) in Increasing Pain Tolerance and Improving the Mental Health of Injured Athletes. Frontiers in psychology, 9, 722. https://doi.org/10.3389/fpsyg.2018.00722\r","\tUssher, M., Spatz, A., Copland, C., Nicolaou, A., Cargill, A., Amini-Tabrizi, N., & McCracken, L. M. (2014). Immediate effects of a brief mindfulness-based body scan on patients with chronic pain. Journal of behavioral medicine, 37(1), 127–134. https://doi.org/10.1007/s10865-012-9466-5\r","\t4. Warren, J. M., Smith, N., & Ashwell, M. (2017). A structured literature review on the role of mindfulness, mindful eating and intuitive eating in changing eating behaviours: Effectiveness and associated potential mechanisms. Nutrition Research Reviews, 30(2), 272-283. doi:10.1017/S0954422417000154 [doi]\r","\t5. Nien, J. T., Wu, C. H., Yang, K. T., Cho, Y. M., Chu, C. H., Chang, Y. K., & Zhou, C. (2020). Mindfulness Training Enhances Endurance Performance and Executive Functions in Athletes: An Event-Related Potential Study. Neural plasticity, 2020, 8213710. https://doi.org/10.1155/2020/8213710\r","\t6. Cropley, M., Ussher, M., & Charitou, E. (2007). Acute effects of a guided relaxation routine (body scan) on tobacco withdrawal symptoms and cravings in abstinent smokers. Addiction (Abingdon, England), 102(6), 989–993. https://doi.org/10.1111/j.1360-0443.2007.01832.x\r","\tBrewer, J. A., Mallik, S., Babuscio, T. A., Nich, C., Johnson, H. E., Deleone, C. M., Minnix-Cotton, C. A., Byrne, S. A., Kober, H., Weinstein, A. J., Carroll, K. M., & Rounsaville, B. J. (2011). Mindfulness training for smoking cessation: results from a randomized controlled trial. Drug and alcohol dependence, 119(1-2), 72–80. https://doi.org/10.1016/j.drugalcdep.2011.05.027\r",\t\r,"Berardi, J. (n.d.). Why ‘listen to your body’ and ‘do what works for you’ rarely work — and what to do instead. Retrieved December 13, 2021 from Precision Nutrition. https://www.precisionnutrition.com/listen-to-your-body",None,None
142,"Suttie, J. (2017 November 13). Four Ways Social Support Makes You More Resilient. Greater Good Magazine. https://greatergood.berkeley.edu/article/item/four_ways_social_support_makes_you_more_resilient","Baxter, J., She’ Ierly, S. M., Eby, C., Mason, L., Cortese, C. F., & Hamman, R. F. (1998). Social Network Factors Associated with Perceived Quality of Life: The San Luis Valley Health and Aging Study. Journal of Aging and Health, 10(3), 287–310. https://doi.org/10.1177/089826439801000302","Chan, Y. K., & Lee, R. P. (2006). Network size, social support and happiness in later life: A comparative study of Beijing and Hong Kong. Journal of Happiness Studies, 7(1), 87-112.","Heinze, J. E., Kruger, D. J., Reischl, T. M., Cupal, S., & Zimmerman, M. A. (2015). Relationships Among Disease, Social Support, and Perceived Health: A Lifespan Approach. American journal of community psychology, 56(3-4), 268–279. https://doi.org/10.1007/s10464-015-9758-3","Ozbay, F., Johnson, D. C., Dimoulas, E., Morgan, C. A., Charney, D., & Southwick, S. (2007). Socia

In [100]:
print(type(separated_references_original.loc[1, 'Citation 01']))
separated_references_original.loc[1, 'Citation 01']

<class 'str'>


'Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17[comma] 2022 from PN Level 1 Sleep[comma] Stress Management[comma] and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255'

## Final separated_references

In [360]:
separated_references = separated_references_original.replace(
    '\[comma\]', ',', regex=True).replace('', None).replace(
    '\s\s', None, regex=True).replace('^\t$|^\r$', None, regex=True
    ).dropna(how='all',axis=1)
separated_references.columns = [f'Citation {str(i+1).zfill(2)}' for i in range(len(separated_references.columns))]
separated_references

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10
0,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255","Examine. (2022 January). Sleep characteristics influence glucose regulation the next day . Examine Study Summaries, https://examine.com/members/summaries/issue/january-2022/summary-1lGLqd","Tsereteli, N., Vallat, R., Fernandez-Tajes, J., Delahanty, L. M., Ordovas, J. M., Drew, D. A., Valdes, A. M., Segata, N., Chan, A. T., Wolf, J., Berry, S. E., Walker, M. P., Spector, T. D., & Franks, P. W. (2022). Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions. Diabetologia, 65(2), 356–365. https://doi.org/10.1007/s00125-021-05608-y",None,NaN,NaN,NaN,NaN,NaN,NaN
2,"Zourdos, M.C. (2020). We Out Here Talkin’ About Practice, Man. Monthly Applications in Strength Sport, 4(1), 71-80.",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Helms, E.. (2019). Give Better Guidance than “Just Work Harder”. Monthly Applications in Strength Sport, 3(6), 57-65.",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Eiselt, A. (2019). How the Brain Controls Eating Behavior. Monthly Applications in Strength Sport, 3(12), 58-69.",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1212,"Lennon, D. (Host). (2022 July 13). How Climate Change Impacts Nutrient Status (No. 446) [Audio podcast episode]. In Sigma Nutrition Radio. Sigma Nutrition. https://sigmanutrition.com/episode446/","Hallström, E., Bajzelj, B., Håkansson, N., Sjons, J., Åkesson, A., Wolk, A., & Sonesson, U. (2021). Dietary climate impact: Contribution of foods and dietary patterns by gender and age in a Swedish population. Journal of Cleaner Production, 306, 127189.",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1213,"Lennon, D. (Host). (2022). Nutrition for Women’s Health (No. 445) [Audio podcast episode]. In Sigma Nutrition Radio. Sigma Nutrition. https://sigmanutrition.com/episode445/",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1214,Optimal Aging Portal Blog Team. (2021 February 24). Trees in the city: the roots of health and well-being. McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2021/02/24/trees-in-the-city-the-roots-of-health-and-well-being?sid=f276c79e-05c9-4086-ada5-3079d5b35086&hl=gardening,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1215,Horticultural therapy: Are there benefits to guided gardening?. (2022 August 3). McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2022/08/03/horticultural-therapy-are-there-benefits-to-guided-gardening,"Horticultural therapy for general health in the older adults: A systematic review and meta-analysis. (n.d.). Retrieved August 19, 2022 from McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/full-article/he/horticultural-therapy-general-health-older-adults-systematic-review-meta-42628?hl=horticultural+therapy",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
separated_references.loc[[205, 175]]

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10
205,"\t1. Khoury, B., Lecomte, T., Fortin, G., Masse, M., Therien, P., Bouchard, V., Chapleau, M. A., Paquin, K., & Hofmann, S. G. (2013). Mindfulness-based therapy: a comprehensive meta-analysis. Clinical psychology review, 33(6), 763–771. https://doi.org/10.1016/j.cpr.2013.05.005\r","\t2. Keng, S. L., Smoski, M. J., & Robins, C. J. (2011). Effects of mindfulness on psychological health: a review of empirical studies. Clinical psychology review, 31(6), 1041–1056. https://doi.org/10.1016/j.cpr.2011.04.006\r","\tFriese, M., Messner, C., & Schaffner, Y. (2012). Mindfulness meditation counteracts self-control depletion. Consciousness and Cognition, 21(2), 1016-1022. doi:10.1016/j.concog.2012.01.008 [doi]\r","\t3. Mohammed, W. A., Pappous, A., & Sharma, D. (2018). Effect of Mindfulness Based Stress Reduction (MBSR) in Increasing Pain Tolerance and Improving the Mental Health of Injured Athletes. Frontiers in psychology, 9, 722. https://doi.org/10.3389/fpsyg.2018.00722\r","\tUssher, M., Spatz, A., Copland, C., Nicolaou, A., Cargill, A., Amini-Tabrizi, N., & McCracken, L. M. (2014). Immediate effects of a brief mindfulness-based body scan on patients with chronic pain. Journal of behavioral medicine, 37(1), 127–134. https://doi.org/10.1007/s10865-012-9466-5\r","\t4. Warren, J. M., Smith, N., & Ashwell, M. (2017). A structured literature review on the role of mindfulness, mindful eating and intuitive eating in changing eating behaviours: Effectiveness and associated potential mechanisms. Nutrition Research Reviews, 30(2), 272-283. doi:10.1017/S0954422417000154 [doi]\r","\t5. Nien, J. T., Wu, C. H., Yang, K. T., Cho, Y. M., Chu, C. H., Chang, Y. K., & Zhou, C. (2020). Mindfulness Training Enhances Endurance Performance and Executive Functions in Athletes: An Event-Related Potential Study. Neural plasticity, 2020, 8213710. https://doi.org/10.1155/2020/8213710\r","\t6. Cropley, M., Ussher, M., & Charitou, E. (2007). Acute effects of a guided relaxation routine (body scan) on tobacco withdrawal symptoms and cravings in abstinent smokers. Addiction (Abingdon, England), 102(6), 989–993. https://doi.org/10.1111/j.1360-0443.2007.01832.x\r","\tBrewer, J. A., Mallik, S., Babuscio, T. A., Nich, C., Johnson, H. E., Deleone, C. M., Minnix-Cotton, C. A., Byrne, S. A., Kober, H., Weinstein, A. J., Carroll, K. M., & Rounsaville, B. J. (2011). Mindfulness training for smoking cessation: results from a randomized controlled trial. Drug and alcohol dependence, 119(1-2), 72–80. https://doi.org/10.1016/j.drugalcdep.2011.05.027\r","Berardi, J. (n.d.). Why ‘listen to your body’ and ‘do what works for you’ rarely work — and what to do instead. Retrieved December 13, 2021 from Precision Nutrition. https://www.precisionnutrition.com/listen-to-your-body"
175,"1. [see ""how to use social media for mental health"" in knowledge library]","\t2. Andreassen, C. S., Torsheim, T., & Pallesen, S. (2014). Use of Online Social Network Sites for Personal Purposes at Work: Does it Impair Self-Reported Performance? Comprehensive Psychology. https://doi.org/10.2466/01.21.CP.3.18","\t3. Schmidt, S. J. (2020). Distracted learning: Big problem and golden opportunity. Journal of Food Science Education, 19(4), 278-291.","\tUniversity of Illinois at Urbana-Champaign, News Bureau. (2020 October 14). Distracted learning a big problem, golden opportunity for educators, students. Science Daily. https://www.sciencedaily.com/releases/2020/10/201014140932.htm",\t4. Simon Fraser University. (2019 October 3). Addictive de-vices: How we can unplug from this 21st century epidemic. Science Daily. https://www.sciencedaily.com/releases/2019/10/191003092039.htm,"\tBerthon, P., Pitt, L., & Campbell, C. (2019). Addictive De-Vices: A Public Policy Analysis of Sources and Solutions to Digital Addiction. Journal of Public Policy & Marketing, 38(4), 451–468. https://doi.org/10.1177/

# [failed] Extract Publication 

In [137]:
import re
def extract_source_name(df):
    pattern = r"(?<=\.\s)[\w\s]+(?=[,.])"
    source_name = [title for title in re.findall(pattern, str(df))] if re.search(pattern, str(df)) else None
    return source_name

# Apply the function to each row in the dataframe to create a new column
separated_references.head().apply(extract_source_name, axis=1)
# separated_references.head(5).apply(print, axis=1)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           None
1                                                                                                                                                                                                                                                                                                                                      

In [147]:
import re
import pandas as pd

def extract_source_name(row):
    pattern = r"(?<=\.\s)[\w\s]+(?=[,.])"
    source_name = re.search(pattern, str(row)).group(0) if re.search(pattern, str(row)) else None
    return pd.Series([source_name])

separated_references.apply(extract_source_name, axis=1)

,0
0,None
1,Retrieved January 17
2,Monthly Applications in Strength Sport
3,Monthly Applications in Strength Sport
4,How the Brain Controls Eating Behavior
...,...
1212,In Sigma Nutrition Radio
1213,In Sigma Nutrition Radio
1214,McMaster Optimal Aging Portal
1215,McMaster Optimal Aging Portal


In [150]:
import re
import pandas as pd

def extract_source_name(row):
    pattern = r"(?<=\.\s)[\w\s]+(?=[,.])"
    source_name = [match+'\n' if re.search(pattern, str(row)) else None for match in re.findall(pattern, str(row))]
    return pd.Series([source_name])

separated_references.head().apply(extract_source_name, axis=1)

,0
0,[]
1,"[Retrieved January 17\n, Sleep characteristics influence glucose regulation the next day \n, Examine Study Summaries\n, M\n, M\n, A\n, M\n, T\n, E\n, P\n, D\n, W\n, Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions\n, Diabetologia\n]"
2,"[Monthly Applications in Strength Sport\n, Citation 02 None\nCitation 03 None\nCitation 04 NaN\nCitation 05 NaN\n \n]"
3,"[Monthly Applications in Strength Sport\n, Citation 02 None\nCitation 03 None\nCitation 04 NaN\nCitation 05 NaN\n \n]"
4,"[How the Brain Controls Eating Behavior\n, Monthly Applications in Strength Sport\n, Citation 02 None\nCitation 03 None\nCitation 04 NaN\nCitation 05 NaN\n \n]"
...,...
1212,"[In Sigma Nutrition Radio\n, Sigma Nutrition\n, Journal of Cleaner Production\n, Citation 03 None\nCitation 04 NaN\nCitation 05 NaN\n \n]"
1213,"[In Sigma Nutrition Radio\n, Sigma Nutrition\n]"
1214,[McMaster Optimal Aging Portal\n]
1215,"[McMaster Optimal Aging Portal\n, Retrieved August 19\n]"


In [160]:
import re
import pandas as pd

def extract_source_name(row):
    print(row.values)
    # return pd.concat([pd.Series([source for source in source_name])]).T

separated_references.head().apply(extract_source_name, axis=1)

[None None None nan nan nan nan nan nan nan nan nan nan]
['Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255'
 'Examine. (2022 January). Sleep characteristics influence glucose regulation the next day . Examine Study Summaries, https://examine.com/members/summaries/issue/january-2022/summary-1lGLqd'
 'Tsereteli, N., Vallat, R., Fernandez-Tajes, J., Delahanty, L. M., Ordovas, J. M., Drew, D. A., Valdes, A. M., Segata, N., Chan, A. T., Wolf, J., Berry, S. E., Walker, M. P., Spector, T. D., & Franks, P. W. (2022). Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions. Diabetologia, 65(2), 356–365. https://doi.org/10.1007/s00125-021-05608-y'
 None nan nan nan nan nan nan nan nan nan]
['Zourdos, M.C. (2020). We Out Here Talkin’ About Practice, Man. Monthly Applic

0    None
1    None
2    None
3    None
4    None
dtype: object

In [169]:
import re
import pandas as pd

def extract_source_name(row):
    # pattern = r"(?<=\.\s)[\w\s]+(?=[,.])"
    pattern = r"(?<=[\w\s]\.\s)[\w\s]+(?=[,.])"
    source_name = [match+'\n' if re.search(pattern, str(row.values)) else '' for match in re.findall(pattern, str(row))]
    return pd.concat([pd.Series([source for source in source_name])]).T

separated_references.head().apply(extract_source_name, axis=1)

C:\Users\silvh\AppData\Local\Temp\ipykernel_25016\1844511363.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.concat([pd.Series([source for source in source_name])]).T


,0,1,2,3,4,5,6,7,8,9,10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Examine Study Summaries\n,M\n,M\n,A\n,M\n,T\n,E\n,P\n,D\n,W\n,Diabetologia\n
2,Monthly Applications in Strength Sport\n,Citation 02 None\nCitation 03 None\nCitation 04 NaN\nCitation 05 NaN\n \n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Monthly Applications in Strength Sport\n,Citation 02 None\nCitation 03 None\nCitation 04 NaN\nCitation 05 NaN\n \n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### By column

In [245]:
import re
import pandas as pd

def extract_source_name(row):
    # pattern = r"(?<=\.\s)[\w\s]+(?=[,.])"
    # pattern = r"(?<=[\w\s]\.\s)[\w\s]+(?=[,.])"
    pattern = r"([\w\s]\.\s)([\w\s]+)([\w\s]+), .*"
    # print(row)
    # pattern = r".+\(.+\)\.([\w\s]+)\. ([\w\s]+) ([\w\s]+)"
    # pattern = r'\.\s([\w\s]+),\s\d+,\s\d+'
    replacement_pattern = r'\2'
    # replacement_pattern = 'hello'
    if type(row) == str:
        source_name = re.sub(pattern, replacement_pattern, row) 
    else:
        source_name = ''
    return source_name
    # return pd.Series(source_name)
    # return pd.concat([pd.Series([source for source in source_name])]).T

def extract_publcation(separated_references):

    separated_publications = separated_references.copy()
    for column in separated_publications.columns:
        separated_publications[column] = separated_publications[column].apply(extract_source_name)

    return separated_publications

extract_publcation(separated_references.head())

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10,Citation 11,Citation 12,Citation 13
0,,,,,,,,,,,,,
1,"Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255",Examine. (2022 January). Sleep characteristics influence glucose regulation the next dayExamine Study Summarie,"Tsereteli, N., Vallat, R., Fernandez-Tajes, J., Delahanty, L. M., Ordovas, J. M., Drew, D. A., Valdes, A. M., Segata, N., Chan, A. T., Wolf, J., Berry, S. E., Walker, M. P., Spector, T. D., & Franks, P. W. (2022). Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditionDiabetologi",,,,,,,,,,
2,"Zourdos, M.C. (2020). We Out Here Talkin’ About Practice, MaMonthly Applications in Strength Spor",,,,,,,,,,,,
3,"Helms, E.. (2019). Give Better Guidance than “Just Work Harder”. Monthly Applications in Strength Sport, 3(6), 57-65.",,,,,,,,,,,,
4,"Eiselt, A. (2019). How the Brain Controls Eating BehavioMonthly Applications in Strength Spor",,,,,,,,,,,,


In [246]:
separated_references.head()

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10,Citation 11,Citation 12,Citation 13
0,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255","Examine. (2022 January). Sleep characteristics influence glucose regulation the next day . Examine Study Summaries, https://examine.com/members/summaries/issue/january-2022/summary-1lGLqd","Tsereteli, N., Vallat, R., Fernandez-Tajes, J., Delahanty, L. M., Ordovas, J. M., Drew, D. A., Valdes, A. M., Segata, N., Chan, A. T., Wolf, J., Berry, S. E., Walker, M. P., Spector, T. D., & Franks, P. W. (2022). Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions. Diabetologia, 65(2), 356–365. https://doi.org/10.1007/s00125-021-05608-y",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Zourdos, M.C. (2020). We Out Here Talkin’ About Practice, Man. Monthly Applications in Strength Sport, 4(1), 71-80.",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Helms, E.. (2019). Give Better Guidance than “Just Work Harder”. Monthly Applications in Strength Sport, 3(6), 57-65.",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Eiselt, A. (2019). How the Brain Controls Eating Behavior. Monthly Applications in Strength Sport, 3(12), 58-69.",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [304]:
import re
import pandas as pd

def extract_source_name(row):
    # pattern = r"(?<=\.\s)[\w\s]+(?=[,.])"
    pattern = r"(?<=[\w\s]\.\s)[\w\s]+(?=[,.])"
    # pattern = r'\.\s([\w\s]+),\s\d+\(\d+\)'
    # pattern = r"([\w\s]\.\s)([\w\s]+)([\w\s]+), .*"
    print(row, type(row))
    # pattern = r".+\(.+\)\.([\w\s]+)\. ([\w\s]+) ([\w\s]+)"
    # pattern = r'\.\s([\w\s]+),\s\d+,\s\d+'
    # replacement_pattern = r'\2'
    replacement_pattern = 'hello'
    if (type(row) == str) & (row != None):
        # print('string')
        try:
            source_name = re.match(pattern, row).group(0)
        except:
            source_name = ''
    else:
        source_name = ''
    print(source_name)
    return source_name
    # return pd.Series(source_name)
    # return pd.concat([pd.Series([source for source in source_name])]).T

def extract_publcation(separated_references):

    separated_publications = separated_references.copy()
    for column in separated_publications.columns:
        separated_publications[column] = separated_publications[column].apply(extract_source_name)

    return separated_publications

extract_publcation(separated_references.head())

None <class 'NoneType'>

Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255 <class 'str'>

Zourdos, M.C. (2020). We Out Here Talkin’ About Practice, Man. Monthly Applications in Strength Sport, 4(1), 71-80. <class 'str'>

Helms, E.. (2019). Give Better Guidance than “Just Work Harder”. Monthly Applications in Strength Sport, 3(6), 57-65. <class 'str'>

Eiselt, A. (2019). How the Brain Controls Eating Behavior. Monthly Applications in Strength Sport, 3(12), 58-69. <class 'str'>

None <class 'NoneType'>

Examine. (2022 January). Sleep characteristics influence glucose regulation the next day . Examine Study Summaries, https://examine.com/members/summaries/issue/january-2022/summary-1lGLqd <class 'str'>

None <class 'NoneType'>

None <class 'NoneType'>

None <class 'NoneType'>

None <class 'NoneType'>

Tseret

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10,Citation 11,Citation 12,Citation 13
0,,,,,,,,,,,,,
1,,,,,,,,,,,,,
2,,,,,,,,,,,,,
3,,,,,,,,,,,,,
4,,,,,,,,,,,,,


## Find the regex

In [277]:
row = """Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255"""
# pattern = r'([\w\s]+\.)\s(\(\S+\))\s([\w\s]+)' # capture groups don't work
pattern = r'([\w\s]+\.) \('
# pattern = r'([\w\s]+\.)\s\([\S]\)'
replacement_pattern = r'\1'
# replacement_pattern = r'\2'
# replacement_pattern = r'\3'
re.sub(pattern, replacement_pattern, row)

'Precision Nutrition.n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255'

In [278]:
row = """Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255"""
pattern = r'([\w\s]+\.) \(\S+'
replacement_pattern = r'\1'
# replacement_pattern = r'\2'
# replacement_pattern = r'\3'
re.sub(pattern, replacement_pattern, row)

'Precision Nutrition. Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255'

In [282]:
row = """Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255"""
pattern = r'([\w\s]+\.) \(\S+\)\.'
replacement_pattern = r'\1'
# replacement_pattern = r'\2'
# replacement_pattern = r'\3'
re.sub(pattern, replacement_pattern, row)

'Precision Nutrition. Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255'

In [286]:
row = "Silvia jumps up"
pattern = r'(Silvia) (jumps)'
replacement_pattern = r'\1 hello'
# replacement_pattern = r'\2'
# replacement_pattern = r'\3'
re.sub(pattern, replacement_pattern, row)

'Silvia hello up'

In [287]:
row = """Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255"""
pattern = r'(?<=[\w\s]+\.) \(\S+\)\.'
replacement_pattern = r'\1'
# replacement_pattern = r'\2'
# replacement_pattern = r'\3'
re.sub(pattern, replacement_pattern, row)

error: look-behind requires fixed-width pattern

In [ ]:
import re
import pandas as pd

def extract_source_name(row):
    pattern = r"([\w\s]\.\s)([\w\s]+)([\w\s]+), .*"
    replacement_pattern = r'\2'
    # replacement_pattern = 'hello'
    if type(row) == str:
        source_name = re.sub(pattern, replacement_pattern, row) 
    else:
        source_name = ''
    return source_name
    # return pd.Series(source_name)
    # return pd.concat([pd.Series([source for source in source_name])]).T

def extract_publcation(separated_references):

    separated_publications = separated_references.copy()
    for column in separated_publications.columns:
        separated_publications[column] = separated_publications[column].apply(extract_source_name)

    return separated_publications

extract_publcation(separated_references.head())

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10,Citation 11,Citation 12,Citation 13
0,,,,,,,,,,,,,
1,"Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255",Examine. (2022 January). Sleep characteristics influence glucose regulation the next dayExamine Study Summarie,"Tsereteli, N., Vallat, R., Fernandez-Tajes, J., Delahanty, L. M., Ordovas, J. M., Drew, D. A., Valdes, A. M., Segata, N., Chan, A. T., Wolf, J., Berry, S. E., Walker, M. P., Spector, T. D., & Franks, P. W. (2022). Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditionDiabetologi",,,,,,,,,,
2,"Zourdos, M.C. (2020). We Out Here Talkin’ About Practice, MaMonthly Applications in Strength Spor",,,,,,,,,,,,
3,"Helms, E.. (2019). Give Better Guidance than “Just Work Harder”. Monthly Applications in Strength Sport, 3(6), 57-65.",,,,,,,,,,,,
4,"Eiselt, A. (2019). How the Brain Controls Eating BehavioMonthly Applications in Strength Spor",,,,,,,,,,,,


In [226]:
separated_references

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10,Citation 11,Citation 12,Citation 13
0,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255","Examine. (2022 January). Sleep characteristics influence glucose regulation the next day . Examine Study Summaries, https://examine.com/members/summaries/issue/january-2022/summary-1lGLqd","Tsereteli, N., Vallat, R., Fernandez-Tajes, J., Delahanty, L. M., Ordovas, J. M., Drew, D. A., Valdes, A. M., Segata, N., Chan, A. T., Wolf, J., Berry, S. E., Walker, M. P., Spector, T. D., & Franks, P. W. (2022). Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions. Diabetologia, 65(2), 356–365. https://doi.org/10.1007/s00125-021-05608-y",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Zourdos, M.C. (2020). We Out Here Talkin’ About Practice, Man. Monthly Applications in Strength Sport, 4(1), 71-80.",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Helms, E.. (2019). Give Better Guidance than “Just Work Harder”. Monthly Applications in Strength Sport, 3(6), 57-65.",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Eiselt, A. (2019). How the Brain Controls Eating Behavior. Monthly Applications in Strength Sport, 3(12), 58-69.",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1212,"Lennon, D. (Host). (2022 July 13). How Climate Change Impacts Nutrient Status (No. 446) [Audio podcast episode]. In Sigma Nutrition Radio. Sigma Nutrition. https://sigmanutrition.com/episode446/","Hallström, E., Bajzelj, B., Håkansson, N., Sjons, J., Åkesson, A., Wolk, A., & Sonesson, U. (2021). Dietary climate impact: Contribution of foods and dietary patterns by gender and age in a Swedish population. Journal of Cleaner Production, 306, 127189.",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1213,"Lennon, D. (Host). (2022). Nutrition for Women’s Health (No. 445) [Audio podcast episode]. In Sigma Nutrition Radio. Sigma Nutrition. https://sigmanutrition.com/episode445/",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1214,Optimal Aging Portal Blog Team. (2021 February 24). Trees in the city: the roots of health and well-being. McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2021/02/24/trees-in-the-city-the-roots-of-health-and-well-being?sid=f276c79e-05c9-4086-ada5-3079d5b35086&hl=gardening,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1215,Horticultural therapy: Are there benefits to guided gardening?. (2022 August 3). McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2022/08/03/horticultural-therapy-are-there-benefits-to-guided-gardening,"Horticultural therapy for general health in the older adults: A systematic review and meta-analysis. (n.d.). Retrieved August 19, 2022 from McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/full-article/he/horticultural-therapy-general-health-older-adults-systematic-review-meta-42628?hl=horticultural+therapy",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pd.concat([pd.Series(references_lists[0].iloc[i], index=range(len(references_lists[0].iloc[i]))) for i in range(len(references_lists))], axis=1).T

In [327]:
separated_references.head()

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10,Citation 11,Citation 12,Citation 13
0,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255","Examine. (2022 January). Sleep characteristics influence glucose regulation the next day . Examine Study Summaries, https://examine.com/members/summaries/issue/january-2022/summary-1lGLqd","Tsereteli, N., Vallat, R., Fernandez-Tajes, J., Delahanty, L. M., Ordovas, J. M., Drew, D. A., Valdes, A. M., Segata, N., Chan, A. T., Wolf, J., Berry, S. E., Walker, M. P., Spector, T. D., & Franks, P. W. (2022). Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions. Diabetologia, 65(2), 356–365. https://doi.org/10.1007/s00125-021-05608-y",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Zourdos, M.C. (2020). We Out Here Talkin’ About Practice, Man. Monthly Applications in Strength Sport, 4(1), 71-80.",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Helms, E.. (2019). Give Better Guidance than “Just Work Harder”. Monthly Applications in Strength Sport, 3(6), 57-65.",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Eiselt, A. (2019). How the Brain Controls Eating Behavior. Monthly Applications in Strength Sport, 3(12), 58-69.",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Map references to type 
Failed at using regex to extract publication

In [384]:
import re
import pandas as pd

def classify(text):
    result = pd.Series(dtype=str)
    for index, value in enumerate(text):
        # print(value)
        if (value == '') | (re.match(r'^\r*$', value) != None): 
            result.loc[index] = None
        elif any(s in value for s in [
            'American College of Sports Medicine. (2020)' ,
            'Haff, G. G., & Triplett, N. T. (Eds.). (2015)',
            'Exercise physiology: nutrition, energy, and human performance',
            'McArdle, W. D., Katch, F. I., & Katch, V. L.',
            'The Essentials of Nutrition and Coaching'
            ]):
            result.loc[index] = 'Textbook'
        elif ('Monthly Applications in Strength Sport'.lower() in value.lower()) | ('massmember' in value):
            result.loc[index] = '2 Research Review: Monthly Applications in Strength Sport'
        elif 'Precision Nutrition Research Insider' in value:
            result.loc[index] = '2 Research Review: Precision Nutrition Research Insider'
        elif 'my.precisionnutrition.com/certification' in value:
            result.loc[index] = 'Precision Nutrition Online Course'
        elif 'Precision Nutrition' in value:
            result.loc[index] = 'Other Precision Nutrition'
        elif 'McMaster Optimal Aging Portal' in value:
            result.loc[index] = '2 Research Review: McMaster Optimal Aging Portal'
        elif 'Sigma Nutrition' in value:
            result.loc[index] = 'Podcast: Sigma Nutrition Radio Podcast'
        elif ('Examine Study Summaries' in value) | ('Examine Study Deep Dives' in value):
            result.loc[index] = '2 Research Review: Examine'
        elif ('Examine' in value) & ('Supplement Guides' in value):
            result.loc[index] = 'Textbook: Examine Supplement Guide'
        elif 'Examine.' in value:
            result.loc[index] = 'Website: Examine'
        elif ('Science Daily' in value) | ('ScienceDaily' in value) | ('sciencedaily' in value):
            result.loc[index] = '2 Research Review: Science Daily'
        elif '[Audio podcast episode]' in value:
            result.loc[index] = 'Podcast: Other'
        elif any(s in value for s in [
            'FoodData Central', 'Government of Canada', 
            'U.S. Department of Agriculture Agricultural Research Service', 'usda.gov/',
            'Health Canada', 'World Health Organization', 'healthlinkbc',
            'Government of British Columbia', 'U.S. Food and Drug Administration',
            'myhealth.alberta.ca',
            'Centres for Disease Control and Prevention', 'cdc.gov/'
            ]):
            result.loc[index] = 'Website: Governmentmental'
            # print(value)
        elif any(s in value for s in [
            'Canadian Society for Exercise Physiology', 'ACSM guidelines', 
            'Centers for Disease Control and Prevention',
            'Compendium of Physical Activities Tracking Guide',
            "from Canada's Food Guide", 'Physical Activity Guidelines Advisory Committee'
            ]):
            result.loc[index] = '3 Guidelines'
        elif any(s in value for s in [
            'Mayo Clinic.', 
            'Monash University', 'Cancer Society', 'Institute of Medicine.',
            'Heart and Stroke Foundation',
            'Arthritis Society.', 'Dietiticians of Canada',
            'cancer.gov'
            ]):
            result.loc[index] = 'Website: Other Reputable Source'
        elif any(s in value for s in [
            'Sleep Foundation', 'Kidney Foundation', 'Linus Pauling Institute',
            'strongerbyscience', 'Healthline.', 'Cleveland Clinic', 'Healthline',
            'sportsdietitians.com', ' Greater Good Magazine.',
            'Gizmodo','gutmicrobiotaforhealth.com','berkeley.edu/','Kitchn.',
            'usnews.com/', 'ABC Everyday.', 'canadasafetycouncil.org',
            'Food Safety Authority of Ireland', 'livestrong.com',
            'Barefoot Strong Blog.',"Today's Dietitian", 'Breast Cancer Network',
            'patient.info/', 'apa.org/research/action'
            ]):
            result.loc[index] = 'Website: Other'
        elif ('Retrieved' in value ) & ('from' in value):
            result.loc[index] = 'Website: Other'
        elif any (s in value for s in [
            'Journal of', 'journal of', 
            'ncbi.nlm.nih', '/doi.', '/doi/', 'doi:', 'DOI:',
            'Archives of', ' Journal, ', 'Frontiers in', 'Cochrane',
            'metaanalysis', 'meta-analysis', 'systematic review', 'Systematic review',
            '. Obesity,', '. JAMA','Advance online publication.',
            'Collegium antropologicum,','Science & Sports,', 'Forum of nutrition,',
            'Zhonghua yi xue za zhi,', 'Sports medicine (Auckland, N.Z.)',
            'Human-centric health: Behaviour change and the prevention of non-communicable diseases.',
            'Nutrition Bulletin,','Physiology & behavior,', 'PloS one,',
            '. Appetite,', 'Medicine & Science in Sports & Exercise',
            'BioMedicine,','Advances in nutrition,', 'mydr.com.au',
            '. Sleep health,', '. Danish medical bulletin,', '. Scientific Reports,',
            'usnews.com/', '. Pacific health dialog,', 'mdpi.com/2072-6643/13/6/1935/htm',
            '. Psychiatry (', '. The Gerontologist'
            ]):
            result.loc[index] = '1 Journal Article'
        elif any (s in value.lower() for s in [
            'Atomic Habits'.lower(), 'Power of habit'.lower(), 
            'Tiny Habits'.lower(), "Why zebras don't get ulcers".lower(),
            'Dopamine Nation: Finding Balance in the Age of Indulgence'.lower(),
            'Switch: How to Change Things When Change Is Hard'.lower(),
            'Drive: The Surprising Truth About What Motivates Us'.lower(),
            'National Academies Press'.lower(), 
            ]):
            result.loc[index] = 'Book'
        elif any (s in value.lower() for s in [
            'same as for ', 'energy compensation from exercise',
            'knowledge library'
            ]):
            result.loc[index] = 'Cross-reference'
        elif any (s in value for s in [
            'Prioritize whole foods and minimize processed foods to support health, good eating habits, and weight management',
            '[Description of multi-component exercise:]'
            ]):
            result.loc[index] = None
        else:
            result.loc[index] = 'Other'
            print(text.name, value)

    return result
    

def classify_reference(separated_references):

    classified = separated_references.copy()
    classified = classified.fillna('').apply(classify, axis=1)
    classified.columns = [f'Citation {str(i+1).zfill(2)}' for i in range(len(classified.columns))]

    return classified

classified_references = classify_reference(separated_references)
classified_references

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Precision Nutrition Online Course,2 Research Review: Examine,1 Journal Article,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2 Research Review: Monthly Applications in Strength Sport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2 Research Review: Monthly Applications in Strength Sport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2 Research Review: Monthly Applications in Strength Sport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1212,Podcast: Sigma Nutrition Radio Podcast,1 Journal Article,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1213,Podcast: Sigma Nutrition Radio Podcast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1214,2 Research Review: McMaster Optimal Aging Portal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1215,2 Research Review: McMaster Optimal Aging Portal,2 Research Review: McMaster Optimal Aging Portal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [237]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

classified_references.count(axis=1).sort_values(ascending=False)

205    10
175     6
617     6
179     6
12      6
       ..
123     0
121     0
113     0
103     0
0       0
Length: 1217, dtype: int64

In [385]:
separated_references.loc[[20, 42, 1211]]

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10
20,"same as for focus on health, not appearance",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,"Benatti, F. B., & Ried-Larsen, M. (2015). The Effects of Breaking up Prolonged Sitting Time: A Review of Experimental Studies. Medicine and science in sports and exercise, 47(10), 2053–2061. https://doi.org/10.1249/MSS.0000000000000654\r",None,"Helms, E. (2021 February). Health & Physical Activity: Is Just Lifting Enough? [Video]. Monthly Applications in Strength Sport. https://www.massmember.com/products/mass-subscription/categories/4527583/posts/15196063",None,None,NaN,NaN,NaN,NaN,NaN
1211,"How a stool test can help find colorectal cancer (n.d.). Retrieved August 17, 2022 from Canadian Cancer Society. https://cancer.ca/en/cancer-information/find-cancer-early/get-screened-for-colorectal-cancer/how-a-stool-test-can-help-find-colorectal-cancer","When should I be screened for colorectal cancer? (n.d.). Retrieved August 17, 2022 from Canadian Cancer Society. https://cancer.ca/en/cancer-information/find-cancer-early/get-screened-for-colorectal-cancer/when-should-i-be-screened-for-colorectal-cancer",None,American Cancer Society Guidelines for the Early Detection of Cancer (2022 March 14). American Cancer Society. https://www.cancer.org/healthy/find-cancer-early/american-cancer-society-guidelines-for-the-early-detection-of-cancer.html,"What are the benefits and limitations of regular cancer screening? (n.d.). Retrieved August 17, 2022 from Canadian Cancer Society. https://cancer.ca/en/cancer-information/find-cancer-early/screening-for-cancer/benefits-and-limitations-of-regular-cancer-screening",None,NaN,NaN,NaN,NaN


In [386]:
classified_references.loc[[20, 42, 1211]]

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10
20,Cross-reference,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,1 Journal Article,NaN,2 Research Review: Monthly Applications in Strength Sport,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1211,Website: Other Reputable Source,Website: Other Reputable Source,NaN,Website: Other Reputable Source,Website: Other Reputable Source,NaN,NaN,NaN,NaN,NaN


In [387]:
classified_references.loc[[205, 175]]

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10
205,1 Journal Article,1 Journal Article,1 Journal Article,1 Journal Article,1 Journal Article,1 Journal Article,1 Journal Article,1 Journal Article,1 Journal Article,Other Precision Nutrition
175,Cross-reference,1 Journal Article,1 Journal Article,2 Research Review: Science Daily,2 Research Review: Science Daily,1 Journal Article,NaN,NaN,NaN,NaN


In [388]:
separated_references.loc[[205, 175]]

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10
205,"\t1. Khoury, B., Lecomte, T., Fortin, G., Masse, M., Therien, P., Bouchard, V., Chapleau, M. A., Paquin, K., & Hofmann, S. G. (2013). Mindfulness-based therapy: a comprehensive meta-analysis. Clinical psychology review, 33(6), 763–771. https://doi.org/10.1016/j.cpr.2013.05.005\r","\t2. Keng, S. L., Smoski, M. J., & Robins, C. J. (2011). Effects of mindfulness on psychological health: a review of empirical studies. Clinical psychology review, 31(6), 1041–1056. https://doi.org/10.1016/j.cpr.2011.04.006\r","\tFriese, M., Messner, C., & Schaffner, Y. (2012). Mindfulness meditation counteracts self-control depletion. Consciousness and Cognition, 21(2), 1016-1022. doi:10.1016/j.concog.2012.01.008 [doi]\r","\t3. Mohammed, W. A., Pappous, A., & Sharma, D. (2018). Effect of Mindfulness Based Stress Reduction (MBSR) in Increasing Pain Tolerance and Improving the Mental Health of Injured Athletes. Frontiers in psychology, 9, 722. https://doi.org/10.3389/fpsyg.2018.00722\r","\tUssher, M., Spatz, A., Copland, C., Nicolaou, A., Cargill, A., Amini-Tabrizi, N., & McCracken, L. M. (2014). Immediate effects of a brief mindfulness-based body scan on patients with chronic pain. Journal of behavioral medicine, 37(1), 127–134. https://doi.org/10.1007/s10865-012-9466-5\r","\t4. Warren, J. M., Smith, N., & Ashwell, M. (2017). A structured literature review on the role of mindfulness, mindful eating and intuitive eating in changing eating behaviours: Effectiveness and associated potential mechanisms. Nutrition Research Reviews, 30(2), 272-283. doi:10.1017/S0954422417000154 [doi]\r","\t5. Nien, J. T., Wu, C. H., Yang, K. T., Cho, Y. M., Chu, C. H., Chang, Y. K., & Zhou, C. (2020). Mindfulness Training Enhances Endurance Performance and Executive Functions in Athletes: An Event-Related Potential Study. Neural plasticity, 2020, 8213710. https://doi.org/10.1155/2020/8213710\r","\t6. Cropley, M., Ussher, M., & Charitou, E. (2007). Acute effects of a guided relaxation routine (body scan) on tobacco withdrawal symptoms and cravings in abstinent smokers. Addiction (Abingdon, England), 102(6), 989–993. https://doi.org/10.1111/j.1360-0443.2007.01832.x\r","\tBrewer, J. A., Mallik, S., Babuscio, T. A., Nich, C., Johnson, H. E., Deleone, C. M., Minnix-Cotton, C. A., Byrne, S. A., Kober, H., Weinstein, A. J., Carroll, K. M., & Rounsaville, B. J. (2011). Mindfulness training for smoking cessation: results from a randomized controlled trial. Drug and alcohol dependence, 119(1-2), 72–80. https://doi.org/10.1016/j.drugalcdep.2011.05.027\r","Berardi, J. (n.d.). Why ‘listen to your body’ and ‘do what works for you’ rarely work — and what to do instead. Retrieved December 13, 2021 from Precision Nutrition. https://www.precisionnutrition.com/listen-to-your-body"
175,"1. [see ""how to use social media for mental health"" in knowledge library]","\t2. Andreassen, C. S., Torsheim, T., & Pallesen, S. (2014). Use of Online Social Network Sites for Personal Purposes at Work: Does it Impair Self-Reported Performance? Comprehensive Psychology. https://doi.org/10.2466/01.21.CP.3.18","\t3. Schmidt, S. J. (2020). Distracted learning: Big problem and golden opportunity. Journal of Food Science Education, 19(4), 278-291.","\tUniversity of Illinois at Urbana-Champaign, News Bureau. (2020 October 14). Distracted learning a big problem, golden opportunity for educators, students. Science Daily. https://www.sciencedaily.com/releases/2020/10/201014140932.htm",\t4. Simon Fraser University. (2019 October 3). Addictive de-vices: How we can unplug from this 21st century epidemic. Science Daily. https://www.sciencedaily.com/releases/2019/10/191003092039.htm,"\tBerthon, P., Pitt, L., & Campbell, C. (2019). Addictive De-Vices: A Public Policy Analysis of Sources and Solutions to Digital Addiction. Journal of Public Policy & Marketing, 38(4), 451–468. https://doi.org/10.1177/

## Iteration 2: remove integer in classification

In [400]:
import re
import pandas as pd

def classify(text):
    result = pd.Series(dtype=str)
    for index, value in enumerate(text):
        # print(value)
        if (value == '') | (re.match(r'^\r*$', value) != None): 
            result.loc[index] = None
        elif any(s in value for s in [
            'American College of Sports Medicine. (2020)' ,
            'Haff, G. G., & Triplett, N. T. (Eds.). (2015)',
            'Exercise physiology: nutrition, energy, and human performance',
            'McArdle, W. D., Katch, F. I., & Katch, V. L.',
            'The Essentials of Nutrition and Coaching'
            ]):
            result.loc[index] = 'Textbook'
        elif ('Monthly Applications in Strength Sport'.lower() in value.lower()) | ('massmember' in value):
            result.loc[index] = 'Research Review: Monthly Applications in Strength Sport'
        elif 'Precision Nutrition Research Insider' in value:
            result.loc[index] = 'Research Review: Precision Nutrition Research Insider'
        elif 'my.precisionnutrition.com/certification' in value:
            result.loc[index] = 'Precision Nutrition Online Course'
        elif 'Precision Nutrition' in value:
            result.loc[index] = 'Other Precision Nutrition'
        elif 'McMaster Optimal Aging Portal' in value:
            result.loc[index] = 'Research Review: McMaster Optimal Aging Portal'
        elif 'Sigma Nutrition' in value:
            result.loc[index] = 'Podcast: Sigma Nutrition Radio Podcast'
        elif ('Examine Study Summaries' in value) | ('Examine Study Deep Dives' in value):
            result.loc[index] = 'Research Review: Examine'
        elif ('Examine' in value) & ('Supplement Guides' in value):
            result.loc[index] = 'Textbook: Examine Supplement Guide'
        elif 'Examine.' in value:
            result.loc[index] = 'Website: Examine'
        elif ('Science Daily' in value) | ('ScienceDaily' in value) | ('sciencedaily' in value):
            result.loc[index] = 'Research Review: Science Daily'
        elif '[Audio podcast episode]' in value:
            result.loc[index] = 'Podcast: Other'
        elif any(s in value for s in [
            'FoodData Central', 'Government of Canada', 
            'U.S. Department of Agriculture Agricultural Research Service', 'usda.gov/',
            'Health Canada', 'World Health Organization', 'healthlinkbc',
            'Government of British Columbia', 'U.S. Food and Drug Administration',
            'myhealth.alberta.ca',
            'Centres for Disease Control and Prevention', 'cdc.gov/'
            ]):
            result.loc[index] = 'Website: Governmentmental'
            # print(value)
        elif any(s in value for s in [
            'Canadian Society for Exercise Physiology', 'ACSM guidelines', 
            'Centers for Disease Control and Prevention',
            'Compendium of Physical Activities Tracking Guide',
            "from Canada's Food Guide", 'Physical Activity Guidelines Advisory Committee'
            ]):
            result.loc[index] = 'Guidelines'
        elif any(s in value for s in [
            'Mayo Clinic.', 
            'Monash University', 'Cancer Society', 'Institute of Medicine.',
            'Heart and Stroke Foundation',
            'Arthritis Society.', 'Dietiticians of Canada',
            'cancer.gov'
            ]):
            result.loc[index] = 'Website: Other Reputable Source'
        elif any(s in value for s in [
            'Sleep Foundation', 'Kidney Foundation', 'Linus Pauling Institute',
            'strongerbyscience', 'Healthline.', 'Cleveland Clinic', 'Healthline',
            'sportsdietitians.com', ' Greater Good Magazine.',
            'Gizmodo','gutmicrobiotaforhealth.com','berkeley.edu/','Kitchn.',
            'usnews.com/', 'ABC Everyday.', 'canadasafetycouncil.org',
            'Food Safety Authority of Ireland', 'livestrong.com',
            'Barefoot Strong Blog.',"Today's Dietitian", 'Breast Cancer Network',
            'patient.info/', 'apa.org/research/action'
            ]):
            result.loc[index] = 'Website: Other'
        elif ('Retrieved' in value ) & ('from' in value):
            result.loc[index] = 'Website: Other'
        elif any (s in value for s in [
            'Journal of', 'journal of', 
            'ncbi.nlm.nih', '/doi.', '/doi/', 'doi:', 'DOI:',
            'Archives of', ' Journal, ', 'Frontiers in', 'Cochrane',
            'metaanalysis', 'meta-analysis', 'systematic review', 'Systematic review',
            '. Obesity,', '. JAMA','Advance online publication.',
            'Collegium antropologicum,','Science & Sports,', 'Forum of nutrition,',
            'Zhonghua yi xue za zhi,', 'Sports medicine (Auckland, N.Z.)',
            'Human-centric health: Behaviour change and the prevention of non-communicable diseases.',
            'Nutrition Bulletin,','Physiology & behavior,', 'PloS one,',
            '. Appetite,', 'Medicine & Science in Sports & Exercise',
            'BioMedicine,','Advances in nutrition,', 'mydr.com.au',
            '. Sleep health,', '. Danish medical bulletin,', '. Scientific Reports,',
            'usnews.com/', '. Pacific health dialog,', 'mdpi.com/2072-6643/13/6/1935/htm',
            '. Psychiatry (', '. The Gerontologist'
            ]):
            result.loc[index] = 'Journal Article'
        elif any (s in value.lower() for s in [
            'Atomic Habits'.lower(), 'Power of habit'.lower(), 
            'Tiny Habits'.lower(), "Why zebras don't get ulcers".lower(),
            'Dopamine Nation: Finding Balance in the Age of Indulgence'.lower(),
            'Switch: How to Change Things When Change Is Hard'.lower(),
            'Drive: The Surprising Truth About What Motivates Us'.lower(),
            'National Academies Press'.lower(), 
            ]):
            result.loc[index] = 'Book'
        elif any (s in value.lower() for s in [
            'same as for ', 'energy compensation from exercise',
            'knowledge library'
            ]):
            result.loc[index] = 'Cross-reference'
        elif any (s in value for s in [
            'Prioritize whole foods and minimize processed foods to support health, good eating habits, and weight management',
            '[Description of multi-component exercise:]'
            ]):
            result.loc[index] = None
        else:
            result.loc[index] = 'Other'
            print(text.name, value)

    return result
    

def classify_reference(separated_references):

    classified = separated_references.copy()
    classified = classified.fillna('').apply(classify, axis=1)
    classified.columns = [f'Citation {str(i+1).zfill(2)}' for i in range(len(classified.columns))]

    return classified

classified_references = classify_reference(separated_references)
classified_references

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Precision Nutrition Online Course,Research Review: Examine,Journal Article,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Research Review: Monthly Applications in Strength Sport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Research Review: Monthly Applications in Strength Sport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Research Review: Monthly Applications in Strength Sport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1212,Podcast: Sigma Nutrition Radio Podcast,Journal Article,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1213,Podcast: Sigma Nutrition Radio Podcast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1214,Research Review: McMaster Optimal Aging Portal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1215,Research Review: McMaster Optimal Aging Portal,Research Review: McMaster Optimal Aging Portal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Sort each row so journal articles are at the beginning

## Iteration 1: Able to sort by type only

In [191]:
def hierarchy(reference_type):
    if reference_type == 'Journal':
        return 1
    elif 'Research Review' in str(reference_type):
        return 2
    elif reference_type:
        return 3
    else:
        return 4



def sort_references(classified_references):
    sorted_references = classified_references.copy()
    sorted_references = sorted_references.apply(lambda x: sorted(x, key=lambda y: hierarchy(y)), axis=1)
    sorted_references = pd.concat([pd.Series(reference, index=range(len(sorted_references[0]))) for reference in sorted_references], axis=1).T#.dropna(axis=1, how='all')
    sorted_references.columns = [f'Citation {str(i+1).zfill(2)}' for i in range(len(sorted_references.columns))]
    return sorted_references

sort_references(classified_references).loc[[205, 175]]

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10
205,Journal Article,Journal Article,Journal Article,Journal Article,Journal Article,Journal Article,Journal Article,Journal Article,Journal Article,Other Precision Nutrition
175,Research Review: Science Daily,Research Review: Science Daily,Cross-reference,Journal Article,Journal Article,Journal Article,NaN,NaN,NaN,NaN


## Iteration 2

In [195]:
def hierarchy(reference_type):
    print(reference_type)
    if reference_type == 'Journal':
        return {1: reference_type.name}
    elif 'Research Review' in str(reference_type):
        return {2: reference_type.name}
    elif reference_type:
        return {3: reference_type.name}
    else:
        return {4: reference_type.name}



def sort_references(classified_references):
    sorted_references = classified_references.copy()
    sorted_references = sorted_references.apply(lambda x: sorted(x, key=lambda y: hierarchy(y).keys()), axis=1)
    sorted_references = pd.concat([pd.Series(reference, index=range(len(sorted_references[0]))) for reference in sorted_references], axis=1).T#.dropna(axis=1, how='all')
    sorted_references.columns = [f'Citation {str(i+1).zfill(2)}' for i in range(len(sorted_references.columns))]
    return sorted_references

sort_references(classified_references).loc[[205, 175]]

nan


AttributeError: 'float' object has no attribute 'name'

## Iteration 3

In [200]:
def hierarchy(reference_type):
    if reference_type == 'Journal':
        return {1: reference_type}
    elif 'Research Review' in str(reference_type):
        return {2: reference_type}
    elif reference_type:
        return {3: reference_type}
    else:
        return {4: reference_type}



def sort_references(classified_references):
    sorted_references = classified_references.copy()
    sorted_references = sorted_references.apply(lambda x: sorted(x, key=lambda y: hierarchy(y).keys()), axis=1)
    sorted_references = sorted_references.apply(lambda x: sorted(x, key=lambda y: hierarchy(y).keys()), axis=1)
    # sorted_references = pd.concat([pd.Series(reference, index=range(len(sorted_references[0]))) for reference in sorted_references], axis=1).T#.dropna(axis=1, how='all')
    # sorted_references.columns = [f'Citation {str(i+1).zfill(2)}' for i in range(len(sorted_references.columns))]
    return sorted_references

sort_references(classified_references).loc[[205, 175]]

205    [Journal Article, Journal Article, Journal Article, Journal Article, Journal Article, Journal Article, Journal Article, Journal Article, Journal Article, Other Precision Nutrition]
175                                [Cross-reference, Journal Article, Journal Article, Research Review: Science Daily, Research Review: Science Daily, Journal Article, nan, nan, nan, nan]
dtype: object

In [256]:
classified_references.loc[[205, 175]]

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10
205,Journal Article,Journal Article,Journal Article,Journal Article,Journal Article,Journal Article,Journal Article,Journal Article,Journal Article,Other Precision Nutrition
175,Cross-reference,Journal Article,Journal Article,Research Review: Science Daily,Research Review: Science Daily,Journal Article,NaN,NaN,NaN,NaN


In [257]:
separated_references.loc[[205, 175]]

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10
205,"\t1. Khoury, B., Lecomte, T., Fortin, G., Masse, M., Therien, P., Bouchard, V., Chapleau, M. A.,...","\t2. Keng, S. L., Smoski, M. J., & Robins, C. J. (2011). Effects of mindfulness on psychological...","\tFriese, M., Messner, C., & Schaffner, Y. (2012). Mindfulness meditation counteracts self-contr...","\t3. Mohammed, W. A., Pappous, A., & Sharma, D. (2018). Effect of Mindfulness Based Stress Reduc...","\tUssher, M., Spatz, A., Copland, C., Nicolaou, A., Cargill, A., Amini-Tabrizi, N., & McCracken,...","\t4. Warren, J. M., Smith, N., & Ashwell, M. (2017). A structured literature review on the role ...","\t5. Nien, J. T., Wu, C. H., Yang, K. T., Cho, Y. M., Chu, C. H., Chang, Y. K., & Zhou, C. (2020...","\t6. Cropley, M., Ussher, M., & Charitou, E. (2007). Acute effects of a guided relaxation routin...","\tBrewer, J. A., Mallik, S., Babuscio, T. A., Nich, C., Johnson, H. E., Deleone, C. M., Minnix-C...","Berardi, J. (n.d.). Why ‘listen to your body’ and ‘do what works for you’ rarely work — and what..."
175,"1. [see ""how to use social media for mental health"" in knowledge library]","\t2. Andreassen, C. S., Torsheim, T., & Pallesen, S. (2014). Use of Online Social Network Sites ...","\t3. Schmidt, S. J. (2020). Distracted learning: Big problem and golden opportunity. Journal of ...","\tUniversity of Illinois at Urbana-Champaign, News Bureau. (2020 October 14). Distracted learnin...",\t4. Simon Fraser University. (2019 October 3). Addictive de-vices: How we can unplug from this ...,"\tBerthon, P., Pitt, L., & Campbell, C. (2019). Addictive De-Vices: A Public Policy Analysis of ...",None,None,NaN,NaN


In [281]:
separated_references.head()

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10
0,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Lev...",Examine. (2022 January). Sleep characteristics influence glucose regulation the next day . Exami...,"Tsereteli, N., Vallat, R., Fernandez-Tajes, J., Delahanty, L. M., Ordovas, J. M., Drew, D. A., V...",None,NaN,NaN,NaN,NaN,NaN,NaN
2,"Zourdos, M.C. (2020). We Out Here Talkin’ About Practice, Man. Monthly Applications in Strength ...",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Helms, E.. (2019). Give Better Guidance than “Just Work Harder”. Monthly Applications in Strengt...",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Eiselt, A. (2019). How the Brain Controls Eating Behavior. Monthly Applications in Strength Spor...",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Iteration 4: Map reference type to ranking

In [288]:
def hierarchy(references_types):
    rankings = []
    for reference_type in references_types.values:
        # print(reference_type)
        if 'Journal' in str(reference_type):
            rankings.append(1)
        elif 'Research Review' in str(reference_type):
            rankings.append(2)
        elif 'Guidelines' in str(reference_type):
            rankings.append(3)
        elif type(reference_type) == str:
            rankings.append(4)
        else:
            rankings.append(5)
    return rankings

def sortby_mapping(rankings, separated_references=separated_references):
    sorted_references_list = []
    mappings = []
    for column, rank in rankings.items():
        mappings.append((rank, column))
    sorted_mappings = sorted(mappings, key=lambda x: x[0])
    for rank, column in sorted_mappings:
        sorted_references_list.append(separated_references.loc[rankings.name, column])
    return sorted_references_list
    


def sort_references(classified_references):
    rankings = classified_references.copy().apply(lambda x: hierarchy(x), axis=1)
    rankings = pd.concat([pd.Series(reference, index=range(len(rankings.iloc[0]))) for reference in rankings], axis=1).T
    rankings.columns = [f'Citation {str(i+1).zfill(2)}' for i in range(len(rankings.columns))]
    rankings.index = classified_references.index
    # print(rankings)
    sorted_references = rankings.apply(sortby_mapping, axis=1)
    sorted_references_df = pd.concat([pd.Series(reference, index=range(len(rankings.iloc[0]))) for reference in sorted_references], axis=1).T
    sorted_references_df.columns = [f'Citation {str(i+1).zfill(2)}' for i in range(len(sorted_references_df.columns))]
    print('Shape:',sorted_references_df.shape)
    return sorted_references_df

sorted_references_df = sort_references(classified_references)
sorted_references_df

Shape: (1217, 10)


,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Tsereteli, N., Vallat, R., Fernandez-Tajes, J., Delahanty, L. M., Ordovas, J. M., Drew, D. A., V...",Examine. (2022 January). Sleep characteristics influence glucose regulation the next day . Exami...,"Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Lev...",None,NaN,NaN,NaN,NaN,NaN,NaN
2,"Zourdos, M.C. (2020). We Out Here Talkin’ About Practice, Man. Monthly Applications in Strength ...",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Helms, E.. (2019). Give Better Guidance than “Just Work Harder”. Monthly Applications in Strengt...",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Eiselt, A. (2019). How the Brain Controls Eating Behavior. Monthly Applications in Strength Spor...",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1212,"Hallström, E., Bajzelj, B., Håkansson, N., Sjons, J., Åkesson, A., Wolk, A., & Sonesson, U. (202...","Lennon, D. (Host). (2022 July 13). How Climate Change Impacts Nutrient Status (No. 446) [Audio p...",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1213,"Lennon, D. (Host). (2022). Nutrition for Women’s Health (No. 445) [Audio podcast episode]. In Si...",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1214,Optimal Aging Portal Blog Team. (2021 February 24). Trees in the city: the roots of health and w...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1215,Horticultural therapy: Are there benefits to guided gardening?. (2022 August 3). McMaster Optima...,Horticultural therapy for general health in the older adults: A systematic review and meta-analy...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [289]:
sorted_references_df.loc[1215, 'Citation 02']

'Horticultural therapy for general health in the older adults: A systematic review and meta-analysis. (n.d.). Retrieved August 19, 2022 from McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/full-article/he/horticultural-therapy-general-health-older-adults-systematic-review-meta-42628?hl=horticultural+therapy'

In [370]:
sorted_references_df.loc[152]

Citation 01       Barr, S. B., & Wright, J. C. (2010). Postprandial energy expenditure in whole-food and processed-food meals: implications for daily energy expenditure. Food & nutrition research, 54, 10.3402/fnr.v54i0.5144. https://doi.org/10.3402/fnr.v54i0.5144
Citation 02                  Broxterman, J. (n.d.). Manufactured deliciousness: Why you can’t stop overeating (plus 3 strategies to get control). Retrieved November 21, 2021 from Precision Nutrition. https://www.precisionnutrition.com/why-you-cant-stop-overeating
Citation 03                                                                                                                                           Prioritize whole foods and minimize processed foods to support health, good eating habits, and weight management.
Citation 04                                                                                                                                                                                                     

In [372]:
classified_references.loc[152]

Citation 01    Other Precision Nutrition
Citation 02              Journal Article
Citation 03                          NaN
Citation 04                          NaN
Citation 05                          NaN
Citation 06                          NaN
Citation 07                          NaN
Citation 08                          NaN
Citation 09                          NaN
Citation 10                          NaN
Name: 152, dtype: object

In [ ]:
original_columns = sorted_references_df.columns
sorted_references_df['Knowledge ID #'] = kl['Knowledge ID #']
save_output(
    sorted_references_df.set_index('Knowledge ID #').reset_index(), description='sorted_kl_references',
    csv_path=path, pickle_path=path)

File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/sources/sorted_kl_references_2023-04-24_2304.sav
Time completed: 2023-04-24 23:04:39.031127
	Object saved as pickle
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/sources/sorted_kl_references_2023-04-24_2304.csv
Time completed: 2023-04-24 23:04:39.072654
	DataFrame saved as CSV


In [296]:

sorted_references_df

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10,Knowledge ID #
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,"Tsereteli, N., Vallat, R., Fernandez-Tajes, J., Delahanty, L. M., Ordovas, J. M., Drew, D. A., V...",Examine. (2022 January). Sleep characteristics influence glucose regulation the next day . Exami...,"Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Lev...",None,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2,"Zourdos, M.C. (2020). We Out Here Talkin’ About Practice, Man. Monthly Applications in Strength ...",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
3,"Helms, E.. (2019). Give Better Guidance than “Just Work Harder”. Monthly Applications in Strengt...",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
4,"Eiselt, A. (2019). How the Brain Controls Eating Behavior. Monthly Applications in Strength Spor...",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...
1212,"Hallström, E., Bajzelj, B., Håkansson, N., Sjons, J., Åkesson, A., Wolk, A., & Sonesson, U. (202...","Lennon, D. (Host). (2022 July 13). How Climate Change Impacts Nutrient Status (No. 446) [Audio p...",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1213.0
1213,"Lennon, D. (Host). (2022). Nutrition for Women’s Health (No. 445) [Audio podcast episode]. In Si...",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1214.0
1214,Optimal Aging Portal Blog Team. (2021 February 24). Trees in the city: the roots of health and w...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1215.0
1215,Horticultural therapy: Are there benefits to guided gardening?. (2022 August 3). McMaster Optima...,Horticultural therapy for general health in the older adults: A systematic review and meta-analy...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1216.0


## Iteration 5: Also sort the classified_references dataframe

In [362]:
def hierarchy(references_types):
    rankings = []
    for reference_type in references_types.values:
        # print(reference_type)
        if 'Journal' in str(reference_type):
            rankings.append(1)
        elif 'Research Review' in str(reference_type):
            rankings.append(2)
        elif 'Guidelines' in str(reference_type):
            rankings.append(3)
        elif type(reference_type) == str:
            rankings.append(4)
        else:
            rankings.append(5)
    return rankings

def sortby_mapping(rankings, separated_references):
    sorted_references_list = []
    mappings = []
    for column, rank in rankings.items():
        mappings.append((rank, column))
    sorted_mappings = sorted(mappings, key=lambda x: x[0])
    for rank, column in sorted_mappings:
        sorted_references_list.append(separated_references.loc[rankings.name, column])
    return sorted_references_list
    


def sort_references(classified_references, separated_references):
    rankings = classified_references.copy().apply(lambda x: hierarchy(x), axis=1)
    rankings = pd.concat([pd.Series(reference, index=range(len(rankings.iloc[0]))) for reference in rankings], axis=1).T
    rankings.columns = [f'Citation {str(i+1).zfill(2)}' for i in range(len(rankings.columns))]
    rankings.index = classified_references.index
    # print(rankings)
    sorted_references = rankings.apply(lambda x: sortby_mapping(x, separated_references), axis=1)
    sorted_references_df = pd.concat([pd.Series(reference, index=range(len(rankings.iloc[0]))) for reference in sorted_references], axis=1).T
    sorted_references_df.columns = [f'Citation {str(i+1).zfill(2)}' for i in range(len(sorted_references_df.columns))]

    sorted_classes = rankings.apply(lambda x: sortby_mapping(x, classified_references), axis=1)
    sorted_classes_df = pd.concat([pd.Series(reference, index=range(len(rankings.iloc[0]))) for reference in sorted_classes], axis=1).T
    sorted_classes_df.columns = [f'Citation {str(i+1).zfill(2)}' for i in range(len(sorted_classes_df.columns))]
    print('sorted_references shape:',sorted_references_df.shape)
    return sorted_references_df, sorted_classes_df

sorted_references_df, sorted_classes_df = sort_references(classified_references, separated_references)
sorted_references_df

sorted_references shape: (1217, 10)


,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Tsereteli, N., Vallat, R., Fernandez-Tajes, J., Delahanty, L. M., Ordovas, J. M., Drew, D. A., Valdes, A. M., Segata, N., Chan, A. T., Wolf, J., Berry, S. E., Walker, M. P., Spector, T. D., & Franks, P. W. (2022). Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions. Diabetologia, 65(2), 356–365. https://doi.org/10.1007/s00125-021-05608-y","Examine. (2022 January). Sleep characteristics influence glucose regulation the next day . Examine Study Summaries, https://examine.com/members/summaries/issue/january-2022/summary-1lGLqd","Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255",None,NaN,NaN,NaN,NaN,NaN,NaN
2,"Zourdos, M.C. (2020). We Out Here Talkin’ About Practice, Man. Monthly Applications in Strength Sport, 4(1), 71-80.",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Helms, E.. (2019). Give Better Guidance than “Just Work Harder”. Monthly Applications in Strength Sport, 3(6), 57-65.",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Eiselt, A. (2019). How the Brain Controls Eating Behavior. Monthly Applications in Strength Sport, 3(12), 58-69.",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1212,"Hallström, E., Bajzelj, B., Håkansson, N., Sjons, J., Åkesson, A., Wolk, A., & Sonesson, U. (2021). Dietary climate impact: Contribution of foods and dietary patterns by gender and age in a Swedish population. Journal of Cleaner Production, 306, 127189.","Lennon, D. (Host). (2022 July 13). How Climate Change Impacts Nutrient Status (No. 446) [Audio podcast episode]. In Sigma Nutrition Radio. Sigma Nutrition. https://sigmanutrition.com/episode446/",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1213,"Lennon, D. (Host). (2022). Nutrition for Women’s Health (No. 445) [Audio podcast episode]. In Sigma Nutrition Radio. Sigma Nutrition. https://sigmanutrition.com/episode445/",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1214,Optimal Aging Portal Blog Team. (2021 February 24). Trees in the city: the roots of health and well-being. McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2021/02/24/trees-in-the-city-the-roots-of-health-and-well-being?sid=f276c79e-05c9-4086-ada5-3079d5b35086&hl=gardening,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1215,Horticultural therapy: Are there benefits to guided gardening?. (2022 August 3). McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2022/08/03/horticultural-therapy-are-there-benefits-to-guided-gardening,"Horticultural therapy for general health in the older adults: A systematic review and meta-analysis. (n.d.). Retrieved August 19, 2022 from McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/full-article/he/horticultural-therapy-general-health-older-adults-systematic-review-meta-42628?hl=horticultural+therapy",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [379]:
sorted_classes_df

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10,Knowledge ID #
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,Journal Article,Research Review: Examine,Precision Nutrition Online Course,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2,Research Review: Monthly Applications in Strength Sport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
3,Research Review: Monthly Applications in Strength Sport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
4,Research Review: Monthly Applications in Strength Sport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...
1212,Journal Article,Podcast: Sigma Nutrition Radio Podcast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1213.0
1213,Podcast: Sigma Nutrition Radio Podcast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1214.0
1214,Research Review: McMaster Optimal Aging Portal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1215.0
1215,Research Review: McMaster Optimal Aging Portal,Research Review: McMaster Optimal Aging Portal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1216.0


## Iteration 6: including ranking

In [401]:
def hierarchy(references_types):
    rankings = []
    for reference_type in references_types.values:
        # print(reference_type)
        if 'Journal' in str(reference_type):
            rankings.append(1)
        elif 'Research Review' in str(reference_type):
            rankings.append(2)
        elif 'Guidelines' in str(reference_type):
            rankings.append(3)
        elif type(reference_type) == str:
            rankings.append(4)
        else:
            rankings.append(5)
    return rankings

def sortby_mapping(rankings, separated_references):
    sorted_references_list = []
    mappings = []
    for column, rank in rankings.items():
        mappings.append((rank, column))
    sorted_mappings = sorted(mappings, key=lambda x: x[0])
    for rank, column in sorted_mappings:
        sorted_references_list.append(separated_references.loc[rankings.name, column])
    return sorted_references_list
    


def sort_references(classified_references, separated_references):
    rankings = classified_references.copy().apply(lambda x: hierarchy(x), axis=1)
    rankings = pd.concat([pd.Series(reference, index=range(len(rankings.iloc[0]))) for reference in rankings], axis=1).T
    rankings.columns = [f'Citation {str(i+1).zfill(2)}' for i in range(len(rankings.columns))]
    rankings.index = classified_references.index
    # print(rankings)
    sorted_references = rankings.apply(lambda x: sortby_mapping(x, separated_references), axis=1)
    sorted_references_df = pd.concat([pd.Series(reference, index=range(len(rankings.iloc[0]))) for reference in sorted_references], axis=1).T
    sorted_references_df.columns = [f'Citation {str(i+1).zfill(2)}' for i in range(len(sorted_references_df.columns))]

    sorted_classes = rankings.apply(lambda x: sortby_mapping(x, classified_references), axis=1)
    sorted_classes_df = pd.concat([pd.Series(reference, index=range(len(rankings.iloc[0]))) for reference in sorted_classes], axis=1).T
    sorted_classes_df.columns = [f'Citation {str(i+1).zfill(2)}' for i in range(len(sorted_classes_df.columns))]
    print('sorted_references shape:',sorted_references_df.shape)
    return sorted_references_df, sorted_classes_df

sorted_references_df, sorted_classes_df = sort_references(classified_references, separated_references)
sorted_references_df

sorted_references shape: (1217, 10)


,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Tsereteli, N., Vallat, R., Fernandez-Tajes, J., Delahanty, L. M., Ordovas, J. M., Drew, D. A., Valdes, A. M., Segata, N., Chan, A. T., Wolf, J., Berry, S. E., Walker, M. P., Spector, T. D., & Franks, P. W. (2022). Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions. Diabetologia, 65(2), 356–365. https://doi.org/10.1007/s00125-021-05608-y","Examine. (2022 January). Sleep characteristics influence glucose regulation the next day . Examine Study Summaries, https://examine.com/members/summaries/issue/january-2022/summary-1lGLqd","Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255",None,NaN,NaN,NaN,NaN,NaN,NaN
2,"Zourdos, M.C. (2020). We Out Here Talkin’ About Practice, Man. Monthly Applications in Strength Sport, 4(1), 71-80.",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Helms, E.. (2019). Give Better Guidance than “Just Work Harder”. Monthly Applications in Strength Sport, 3(6), 57-65.",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Eiselt, A. (2019). How the Brain Controls Eating Behavior. Monthly Applications in Strength Sport, 3(12), 58-69.",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1212,"Hallström, E., Bajzelj, B., Håkansson, N., Sjons, J., Åkesson, A., Wolk, A., & Sonesson, U. (2021). Dietary climate impact: Contribution of foods and dietary patterns by gender and age in a Swedish population. Journal of Cleaner Production, 306, 127189.","Lennon, D. (Host). (2022 July 13). How Climate Change Impacts Nutrient Status (No. 446) [Audio podcast episode]. In Sigma Nutrition Radio. Sigma Nutrition. https://sigmanutrition.com/episode446/",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1213,"Lennon, D. (Host). (2022). Nutrition for Women’s Health (No. 445) [Audio podcast episode]. In Sigma Nutrition Radio. Sigma Nutrition. https://sigmanutrition.com/episode445/",None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1214,Optimal Aging Portal Blog Team. (2021 February 24). Trees in the city: the roots of health and well-being. McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2021/02/24/trees-in-the-city-the-roots-of-health-and-well-being?sid=f276c79e-05c9-4086-ada5-3079d5b35086&hl=gardening,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1215,Horticultural therapy: Are there benefits to guided gardening?. (2022 August 3). McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2022/08/03/horticultural-therapy-are-there-benefits-to-guided-gardening,"Horticultural therapy for general health in the older adults: A systematic review and meta-analysis. (n.d.). Retrieved August 19, 2022 from McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/full-article/he/horticultural-therapy-general-health-older-adults-systematic-review-meta-42628?hl=horticultural+therapy",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [390]:
sorted_classes_df

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1 Journal Article,2 Research Review: Examine,Precision Nutrition Online Course,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2 Research Review: Monthly Applications in Strength Sport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2 Research Review: Monthly Applications in Strength Sport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2 Research Review: Monthly Applications in Strength Sport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1212,1 Journal Article,Podcast: Sigma Nutrition Radio Podcast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1213,Podcast: Sigma Nutrition Radio Podcast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1214,2 Research Review: McMaster Optimal Aging Portal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1215,2 Research Review: McMaster Optimal Aging Portal,2 Research Review: McMaster Optimal Aging Portal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Melt the dataframes so there is 1 reference per row

In [402]:
sorted_references_df['Knowledge ID #'] = kl['Knowledge ID #']
melted_sorted_references_df = sorted_references_df.melt(
    id_vars=['Knowledge ID #'],
    var_name='Citation #', value_name='Reference',
    ignore_index=False
).sort_values(['Knowledge ID #', 'Citation #'])
print(melted_sorted_references_df.shape)
melted_sorted_references_df 

(12170, 3)


,Knowledge ID #,Citation #,Reference
0,1.0,Citation 01,NaN
0,1.0,Citation 02,NaN
0,1.0,Citation 03,NaN
0,1.0,Citation 04,NaN
0,1.0,Citation 05,NaN
...,...,...,...
1216,1217.0,Citation 06,NaN
1216,1217.0,Citation 07,NaN
1216,1217.0,Citation 08,NaN
1216,1217.0,Citation 09,NaN


In [403]:
sorted_classes_df.loc[[205, 175]]

,Citation 01,Citation 02,Citation 03,Citation 04,Citation 05,Citation 06,Citation 07,Citation 08,Citation 09,Citation 10
205,Journal Article,Journal Article,Journal Article,Journal Article,Journal Article,Journal Article,Journal Article,Journal Article,Journal Article,Other Precision Nutrition
175,Journal Article,Journal Article,Journal Article,Research Review: Science Daily,Research Review: Science Daily,Cross-reference,NaN,NaN,NaN,NaN


In [404]:
sorted_classes_df['Knowledge ID #'] = kl['Knowledge ID #']
melted_sorted_classes_df = sorted_classes_df.melt(
    id_vars=['Knowledge ID #'],
    var_name='Citation #', value_name='Reference Type',
    ignore_index=False
).sort_values(['Knowledge ID #', 'Citation #'])
print(melted_sorted_classes_df.shape)
melted_sorted_classes_df.head(20)

(12170, 3)


,Knowledge ID #,Citation #,Reference Type
0,1.0,Citation 01,NaN
0,1.0,Citation 02,NaN
0,1.0,Citation 03,NaN
0,1.0,Citation 04,NaN
0,1.0,Citation 05,NaN
0,1.0,Citation 06,NaN
0,1.0,Citation 07,NaN
0,1.0,Citation 08,NaN
0,1.0,Citation 09,NaN
0,1.0,Citation 10,NaN


In [405]:
print(melted_sorted_references_df['Reference'].isna().sum())
print(melted_sorted_classes_df['Reference Type'].isna().sum())

10044
10046


# Merge the references with the reference types

In [406]:
merged_df = melted_sorted_references_df.merge(melted_sorted_classes_df, how='outer')
print(merged_df.shape)
merged_df.tail(20)

(12170, 4)


,Knowledge ID #,Citation #,Reference,Reference Type
12150,1216.0,Citation 01,Horticultural therapy: Are there benefits to guided gardening?. (2022 August 3). McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2022/08/03/horticultural-therapy-are-there-benefits-to-guided-gardening,Research Review: McMaster Optimal Aging Portal
12151,1216.0,Citation 02,"Horticultural therapy for general health in the older adults: A systematic review and meta-analysis. (n.d.). Retrieved August 19, 2022 from McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/full-article/he/horticultural-therapy-general-health-older-adults-systematic-review-meta-42628?hl=horticultural+therapy",Research Review: McMaster Optimal Aging Portal
12152,1216.0,Citation 03,None,NaN
12153,1216.0,Citation 04,NaN,NaN
12154,1216.0,Citation 05,NaN,NaN
12155,1216.0,Citation 06,NaN,NaN
12156,1216.0,Citation 07,NaN,NaN
12157,1216.0,Citation 08,NaN,NaN
12158,1216.0,Citation 09,NaN,NaN
12159,1216.0,Citation 10,NaN,NaN


In [418]:
merged_df['Citation #'] = merged_df['Citation #'].str.replace('Citation ', '').astype(int)
merged_df.tail(20)

,Knowledge ID #,Citation #,Reference,Reference Type,Reference Rank
12150,1216.0,1,Horticultural therapy: Are there benefits to guided gardening?. (2022 August 3). McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2022/08/03/horticultural-therapy-are-there-benefits-to-guided-gardening,Research Review: McMaster Optimal Aging Portal,2.0
12151,1216.0,2,"Horticultural therapy for general health in the older adults: A systematic review and meta-analysis. (n.d.). Retrieved August 19, 2022 from McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/full-article/he/horticultural-therapy-general-health-older-adults-systematic-review-meta-42628?hl=horticultural+therapy",Research Review: McMaster Optimal Aging Portal,2.0
12152,1216.0,3,None,NaN,NaN
12153,1216.0,4,NaN,NaN,NaN
12154,1216.0,5,NaN,NaN,NaN
12155,1216.0,6,NaN,NaN,NaN
12156,1216.0,7,NaN,NaN,NaN
12157,1216.0,8,NaN,NaN,NaN
12158,1216.0,9,NaN,NaN,NaN
12159,1216.0,10,NaN,NaN,NaN


### Check for match

In [407]:
filter = (merged_df['Reference'].isna() & merged_df['Reference Type'].notna()) | (merged_df['Reference'].notna() & merged_df['Reference Type'].isna())
merged_df[filter]
# These are not references, so that's fine

,Knowledge ID #,Citation #,Reference,Reference Type
1522,153.0,Citation 03,"Prioritize whole foods and minimize processed foods to support health, good eating habits, and weight management.",NaN
1645,165.0,Citation 06,[Description of multi-component exercise:],NaN


## Drop null rows

In [409]:
merged_df.dropna(subset=['Reference Type'])

,Knowledge ID #,Citation #,Reference,Reference Type
10,2.0,Citation 01,"Tsereteli, N., Vallat, R., Fernandez-Tajes, J., Delahanty, L. M., Ordovas, J. M., Drew, D. A., Valdes, A. M., Segata, N., Chan, A. T., Wolf, J., Berry, S. E., Walker, M. P., Spector, T. D., & Franks, P. W. (2022). Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions. Diabetologia, 65(2), 356–365. https://doi.org/10.1007/s00125-021-05608-y",Journal Article
11,2.0,Citation 02,"Examine. (2022 January). Sleep characteristics influence glucose regulation the next day . Examine Study Summaries, https://examine.com/members/summaries/issue/january-2022/summary-1lGLqd",Research Review: Examine
12,2.0,Citation 03,"Precision Nutrition. (n.d.). Sleep 101: Why do we sleep?. Retrieved January 17, 2022 from PN Level 1 Sleep, Stress Management, and Recovery Coaching Certification https://my.precisionnutrition.com/certification#/448583/card/139028255",Precision Nutrition Online Course
20,3.0,Citation 01,"Zourdos, M.C. (2020). We Out Here Talkin’ About Practice, Man. Monthly Applications in Strength Sport, 4(1), 71-80.",Research Review: Monthly Applications in Strength Sport
30,4.0,Citation 01,"Helms, E.. (2019). Give Better Guidance than “Just Work Harder”. Monthly Applications in Strength Sport, 3(6), 57-65.",Research Review: Monthly Applications in Strength Sport
...,...,...,...,...
12140,1215.0,Citation 01,Optimal Aging Portal Blog Team. (2021 February 24). Trees in the city: the roots of health and well-being. McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2021/02/24/trees-in-the-city-the-roots-of-health-and-well-being?sid=f276c79e-05c9-4086-ada5-3079d5b35086&hl=gardening,Research Review: McMaster Optimal Aging Portal
12150,1216.0,Citation 01,Horticultural therapy: Are there benefits to guided gardening?. (2022 August 3). McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2022/08/03/horticultural-therapy-are-there-benefits-to-guided-gardening,Research Review: McMaster Optimal Aging Portal
12151,1216.0,Citation 02,"Horticultural therapy for general health in the older adults: A systematic review and meta-analysis. (n.d.). Retrieved August 19, 2022 from McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/full-article/he/horticultural-therapy-general-health-older-adults-systematic-review-meta-42628?hl=horticultural+therapy",Research Review: McMaster Optimal Aging Portal
12160,1217.0,Citation 01,Optimal Aging Portal Blog Team. (2019 Feb 27). Cultivating health and well-being. McMaster Optimal Aging Portal. https://www.mcmasteroptimalaging.org/blog/detail/blog/2019/02/27/cultivating-health-and-well-being?sid=f276c79e-05c9-4086-ada5-3079d5b35086&hl=gardening\r,Research Review: McMaster Optimal Aging Portal


# Add column with reference type rank

In [422]:
def add_reference_rank(reference_type):
    if 'Journal' in str(reference_type):
        return 1
    elif 'Research Review' in str(reference_type):
        return 2
    elif 'Guidelines' in str(reference_type):
        return 3
    elif type(reference_type) == str:
        return 4
    
merged_df['Reference Rank'] = merged_df['Reference Type'].apply(lambda x: add_reference_rank(x))
merged_df.head(20)

,Knowledge ID #,Citation #,Reference,Reference Type,Reference Rank
0,1.0,1,NaN,NaN,NaN
1,1.0,2,NaN,NaN,NaN
2,1.0,3,NaN,NaN,NaN
3,1.0,4,NaN,NaN,NaN
4,1.0,5,NaN,NaN,NaN
5,1.0,6,NaN,NaN,NaN
6,1.0,7,NaN,NaN,NaN
7,1.0,8,NaN,NaN,NaN
8,1.0,9,NaN,NaN,NaN
9,1.0,10,NaN,NaN,NaN


# Sort by reference type rank

In [425]:
final_df = merged_df.dropna(subset=['Reference Type']).sort_values(['Reference Rank', 'Reference Type'])
final_df['Reference Rank'] = final_df['Reference Rank'].astype(int)
final_df['Knowledge ID #'] = final_df['Knowledge ID #'].astype(int)
final_df

,Knowledge ID #,Citation #,Reference,Reference Type,Reference Rank
10,2,1,"Tsereteli, N., Vallat, R., Fernandez-Tajes, J., Delahanty, L. M., Ordovas, J. M., Drew, D. A., Valdes, A. M., Segata, N., Chan, A. T., Wolf, J., Berry, S. E., Walker, M. P., Spector, T. D., & Franks, P. W. (2022). Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions. Diabetologia, 65(2), 356–365. https://doi.org/10.1007/s00125-021-05608-y",Journal Article,1
50,6,1,"Paravlic, A. H., Slimani, M., Tod, D., Marusic, U., Milanovic, Z., & Pisot, R. (2018). Effects and Dose-Response Relationships of Motor Imagery Practice on Strength Development in Healthy Adult Populations: a Systematic Review and Meta-analysis. Sports medicine (Auckland, N.Z.), 48(5), 1165–1187. https://doi.org/10.1007/s40279-018-0874-8",Journal Article,1
70,8,1,"Jäger, R., Kerksick, C. M., Campbell, B. I., Cribb, P. J., Wells, S. D., Skwiat, T. M., ... & Antonio, J. (2017). International society of sports nutrition position stand: protein and exercise. Journal of the International Society of Sports Nutrition, 14(1), 1-25.",Journal Article,1
90,10,1,"Pontzer, H., Yamada, Y., Sagayama, H., Ainslie, P. N., Andersen, L. F., Anderson, L. J., Arab, L., Baddou, I., Bedu-Addo, K., Blaak, E. E., Blanc, S., Bonomi, A. G., Bouten, C., Bovet, P., Buchowski, M. S., Butte, N. F., Camps, S. G., Close, G. L., Cooper, J. A., Cooper, R., … IAEA DLW Database Consortium (2021). Daily energy expenditure through the human life course. Science (New York, N.Y.), 373(6556), 808–812. https://doi.org/10.1126/science.abe5017 [https://pubmed.ncbi.nlm.nih.gov/34385400/]",Journal Article,1
100,11,1,"Sinaki, M., & Mikkelsen, B. A. (1984). Postmenopausal spinal osteoporosis: flexion versus extension exercises. Archives of physical medicine and rehabilitation, 65(10), 593–596.",Journal Article,1
...,...,...,...,...,...
12102,1211,3,American Cancer Society Guidelines for the Early Detection of Cancer (2022 March 14). American Cancer Society. https://www.cancer.org/healthy/find-cancer-early/american-cancer-society-guidelines-for-the-early-detection-of-cancer.html,Website: Other Reputable Source,4
12110,1212,1,"How a stool test can help find colorectal cancer (n.d.). Retrieved August 17, 2022 from Canadian Cancer Society. https://cancer.ca/en/cancer-information/find-cancer-early/get-screened-for-colorectal-cancer/how-a-stool-test-can-help-find-colorectal-cancer",Website: Other Reputable Source,4
12111,1212,2,"When should I be screened for colorectal cancer? (n.d.). Retrieved August 17, 2022 from Canadian Cancer Society. https://cancer.ca/en/cancer-information/find-cancer-early/get-screened-for-colorectal-cancer/when-should-i-be-screened-for-colorectal-cancer",Website: Other Reputable Source,4
12112,1212,3,American Cancer Society Guidelines for the Early Detection of Cancer (2022 March 14). American Cancer Society. https://www.cancer.org/healthy/find-cancer-early/american-cancer-society-guidelines-for-the-early-detection-of-cancer.html,Website: Other Reputable Source,4


In [427]:
save_output(
    final_df,
    description = 'ranked_kl_references',
    csv_path=path, pickle_path=path
)

File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/sources/ranked_kl_references_2023-04-25_0058.sav
Time completed: 2023-04-25 00:58:40.654838
	Object saved as pickle
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/sources/ranked_kl_references_2023-04-25_0058.csv
Time completed: 2023-04-25 00:58:40.722851
	DataFrame saved as CSV


# *End of Page*